# **(1.0)** Metabolic Control Analysis in the Context of Large Metabolic Networks

A common application of metabolic modeling and engineering is to increase the yield of a specific metabolite from a microbial culture, plant, etc. A key difficulty here is that the organism you're working with is not just a cell or set of cells that contain a linear path from some carbon/energy source to the metabolite you care about. There is, to put it simply, a lot of other stuff going on metabolically, some of which will divert matter and energy away from the product of interest. Let's look at the following pathway that we're interested in ...

![alt text](MCA_Network.png "Title")

This should look familiar from Day Two's notebook. However, we'll see that there's more than meets the eye in this case. Let's see a worked example.

## **(1.1)** Worked Example

<div class="alert alert-block alert-info">

**Instructions:** Run the three cells below. For the set of provided initial conditions, determine the control coefficient for all enzymes as we did before. Do the results make sense? Do they sum up to 1? Once all groups have their answers, let's discuss as a whole class.

</div>

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------
# In this cell, we are importing the Python packages we'll need for today's exercises. 
#-----------------------------------------------------------------------------------------------------------------------

import numpy as np                  
import math                                                                  
import seaborn as sns               
import matplotlib.pyplot as plt     
import ipywidgets as widgets        
from IPython.display import display, Markdown, clear_output            
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots 
import pandas as pd                
from scipy import interpolate       
%matplotlib inline                  
sns.set()                           

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------
# In this cell we are defining functions that we will need during our simulation.
#-----------------------------------------------------------------------------------------------------------------------

def evaluate_flux_wEnzyme(S, K, enzyme):                           

    '''
    DESCRIPTION
    
    Takes a rate constant k and a substrate concentration S 
    and returns a flux value.
    
    INPUTS
    
    k = a numeric first order rate constant
    
    S = a numeric concentration of a substrate
    
    enzyme = a numeric concentration of an enzyme
    
    OUTPUTS
    
    flux = flux, or reaction rate, given substrate 
    concentration, first order rate constant, and
    enzyme concentration
    '''

    flux=K*S*enzyme
    return flux

def evaluate_dLAdt_f(Jsa, Jab, Jau, f_s, f_a):
    '''
    DESCRIPTION
    
    Takes the fluxes into out and out of metabolite A (Jsa and Jab)
    in the forward directionand the fractional labeling of S and A 
    to calculate the change in concentration of either labeled or unlabeled A.
    
    INPUTS
    
    Jsa = the numeric flux value Jsa (flux from S to A)
    
    Jab = the numeric flux value Jab (flux from A to B)
    
    f_s = a float value from 0 to 1 representing the fractional labeling of S
    
    f_a = a float value from 0 to 1 representing the fractional labeling of A
    
    OUTPUTS
    
    dLAdt = the change in the labeled or unlabeled concentration
    of A
    
    '''
    
    dLAdt = Jsa * f_s - Jab * f_a - Jau * f_a
    return dLAdt

def evaluate_dLAdt_b(Jsa_b, Jab_b, f_a, f_b):
    
    dLAdt = - Jsa_b * f_a + Jab_b * f_b
    return dLAdt

def evaluate_dLBdt_f(Jab, Jbp, f_a, f_b):
    dLBdt = Jab * f_a - Jbp * f_b
    return dLBdt

def evaluate_dLBdt_b(Jab_b, Jbp_b, f_b, f_p):
    dLBdt = - Jab_b * f_b + Jbp_b * f_p
    return dLBdt

def evaluate_dLPdt_f(Jbp, Jpx, f_b, f_p):                             
    dLPdt = Jbp * f_b - Jpx * f_p
    return dLPdt

def evaluate_dLPdt_b(Jbp_b, f_p):
    dLPdt = - Jbp_b * f_p
    return dLPdt

def evaluate_fractional_labeling(current_labeled, current_total): 
    if current_total == 0:                                        
        f = 0                                                     
    else:                                                         
        f = current_labeled / current_total                                                                         
    return f

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------
# Here we define the simulation function we need to run MCA.
#-----------------------------------------------------------------------------------------------------------------------

def MCA(total_time = 10000, timestep = 1, S = 10, A = 0, B = 0, P = 0, LS = 0, LA = 0, LB = 0,
        LP = 0, K_SA = 0.1, K_SA_b = 0.1, K_AB = 0.1, K_AB_b = 0.01,
        K_BP = 0.1, K_BP_b = 0.01, K_PX = 0.1, K_AU = 1, enzyme_SA = 1, enzyme_AB = 1, 
        enzyme_BP = 1, enzyme_PX = 1, enzyme_AU = 1, mode = 'flux'): 
       
    times = np.arange(0, total_time, timestep)                   
    
    S_total = S + LS                                             
    A_total = A + LA                                             
    B_total = B + LB
    P_total = P + LP
    
    S_list = [S]                                                                                        
    A_list = [A]                                                                                     
    B_list = [B]                                      
    P_list = [P]                                                                                    
                                                      
    LS_list = [LS]                                    
    LA_list = [LA]
    LB_list = [LB]
    LP_list = [LP]
        
    S_total_list = [S_total]                          
    A_total_list = [A_total]
    B_total_list = [B_total]
    P_total_list = [P_total]
    
    fs_list = [evaluate_fractional_labeling(LS, S_total)]       
    fa_list = [evaluate_fractional_labeling(LA, A_total)]       
    fb_list = [evaluate_fractional_labeling(LB, B_total)]       
    fp_list = [evaluate_fractional_labeling(LP, P_total)]
    
    jsa_list = [evaluate_flux_wEnzyme(S_total_list[-1], K_SA,enzyme_SA)]          
    jsa_b_list = [evaluate_flux_wEnzyme(A_total_list[-1], K_SA_b,enzyme_SA)]
    jab_list = [evaluate_flux_wEnzyme(A_total_list[-1], K_AB,enzyme_AB)]
    jab_b_list = [evaluate_flux_wEnzyme(B_total_list[-1], K_AB_b,enzyme_AB)]
    jbp_list = [evaluate_flux_wEnzyme(B_total_list[-1], K_BP,enzyme_BP)]
    jbp_b_list = [evaluate_flux_wEnzyme(P_total_list[-1], K_BP_b,enzyme_BP)]
    jpx_list = [evaluate_flux_wEnzyme(P_total_list[-1], K_PX,enzyme_PX)]
    jau_list = [evaluate_flux_wEnzyme(A_total_list[-1], K_AU,enzyme_AU)]
    
    for i in times:                                  
        
        current_S = S                                
        current_LS = LS                              
    
        current_Jsa = evaluate_flux_wEnzyme(S_total_list[-1], K_SA, enzyme_SA)          
        current_Jsa_b = evaluate_flux_wEnzyme(A_total_list[-1], K_SA_b, enzyme_SA)      
        current_Jab = evaluate_flux_wEnzyme(A_total_list[-1], K_AB, enzyme_AB)          
        current_Jab_b = evaluate_flux_wEnzyme(B_total_list[-1], K_AB_b, enzyme_AB)
        current_Jbp = evaluate_flux_wEnzyme(B_total_list[-1], K_BP, enzyme_BP)
        current_Jbp_b = evaluate_flux_wEnzyme(P_total_list[-1], K_BP_b, enzyme_BP)
        current_Jpx = evaluate_flux_wEnzyme(P_total_list[-1], K_PX, enzyme_PX)
        current_Jau = evaluate_flux_wEnzyme(A_total_list[-1], K_AU, enzyme_AU)
        
        current_A = A_list[-1] + (evaluate_dLAdt_f(current_Jsa, current_Jab, current_Jau, (1 - fs_list[-1]), (1 - fa_list[-1])) * timestep)
        current_B = B_list[-1] + (evaluate_dLBdt_f(current_Jab, current_Jbp, (1 - fa_list[-1]), (1 - fb_list[-1])) * timestep)
        current_P = P_list[-1] + (evaluate_dLPdt_f(current_Jbp, current_Jpx, (1 - fb_list[-1]), (1 - fp_list[-1])) * timestep)
        current_LA = LA_list[-1] + (evaluate_dLAdt_f(current_Jsa, current_Jab, current_Jau, fs_list[-1], fa_list[-1]) * timestep)
        current_LB = LB_list[-1] + (evaluate_dLBdt_f(current_Jab, current_Jbp, fa_list[-1], fb_list[-1]) * timestep)
        current_LP = LP_list[-1] + (evaluate_dLPdt_f(current_Jbp, current_Jpx, fb_list[-1], fp_list[-1]) * timestep)
        
        current_S_total = current_S + current_LS
        current_A_total = current_A + current_LA
        current_B_total = current_B + current_LB
        current_P_total = current_P + current_LP        
        current_fs = evaluate_fractional_labeling(current_LS, current_S_total)
        current_fa = evaluate_fractional_labeling(current_LA, current_A_total)
        current_fb = evaluate_fractional_labeling(current_LB, current_B_total)
        current_fp = evaluate_fractional_labeling(current_LP, current_P_total)        
                
        current_A = current_A + (evaluate_dLAdt_b(current_Jsa_b, current_Jab_b, (1 - fa_list[-1]), (1 - fb_list[-1])) * timestep)
        current_B = current_B + (evaluate_dLBdt_b(current_Jab_b, current_Jbp_b, (1 - fb_list[-1]), (1 - fp_list[-1])) * timestep)
        current_P = current_P + (evaluate_dLPdt_b(current_Jbp_b, (1 - fp_list[-1])) * timestep)
        current_LA = current_LA + (evaluate_dLAdt_b(current_Jsa_b, current_Jab_b, fa_list[-1], fb_list[-1]) * timestep)
        current_LB = current_LB + (evaluate_dLBdt_b(current_Jab_b, current_Jbp_b, fb_list[-1], fp_list[-1]) * timestep)
        current_LP = current_LP + (evaluate_dLPdt_b(current_Jbp_b, fp_list[-1]) * timestep)
            
        current_S_total = current_S + current_LS
        current_A_total = current_A + current_LA
        current_B_total = current_B + current_LB
        current_P_total = current_P + current_LP 
        current_fs = evaluate_fractional_labeling(current_LS, current_S_total)
        current_fa = evaluate_fractional_labeling(current_LA, current_A_total)
        current_fb = evaluate_fractional_labeling(current_LB, current_B_total)
        current_fp = evaluate_fractional_labeling(current_LP, current_P_total)    
        
        LS_list.append(current_LS)
        LA_list.append(current_LA)
        LB_list.append(current_LB)
        LP_list.append(current_LP)

        S_list.append(current_S)
        A_list.append(current_A)
        B_list.append(current_B)
        P_list.append(current_P)

        S_total_list.append(current_S_total)
        A_total_list.append(current_A_total)
        B_total_list.append(current_B_total)
        P_total_list.append(current_P_total)

        fs_list.append(current_fs)
        fa_list.append(current_fa)
        fb_list.append(current_fb)
        fp_list.append(current_fp)
        
        jsa_list.append(current_Jsa)
        jsa_b_list.append(current_Jsa_b)
        jab_list.append(current_Jab)
        jab_b_list.append(current_Jab_b)
        jbp_list.append(current_Jbp)
        jbp_b_list.append(current_Jbp_b)
        jpx_list.append(current_Jpx)
        jau_list.append(current_Jau)
    
    jsa_net = np.array(jsa_list) - np.array(jsa_b_list)              
    jab_net = np.array(jab_list) - np.array(jab_b_list)              
    jbp_net = np.array(jbp_list) - np.array(jbp_b_list)              
    jpx_net = np.array(jpx_list)  
    jau_net = np.array(jau_list)

    times_including_initial = np.insert(times, 0, times[0] - timestep) 

    if mode == 'no_au':
        
        print('The flux through the pathway is:', jpx_list[-1])
        
    if mode == 'flux':
        fig, ax = plt.subplots(1, 1, figsize = (12, 5))   
        ax.plot(times_including_initial, jsa_net, label = 'S -> A; {0}'.format(enzyme_SA))
        ax.plot(times_including_initial, jab_net, label = 'A -> B; {0}'.format(enzyme_AB))
        ax.plot(times_including_initial, jbp_net, label = 'B -> P; {0}'.format(enzyme_BP))  
        ax.plot(times_including_initial, jpx_net, label = 'P -> X; {0}'.format(enzyme_PX))
        ax.plot(times_including_initial, jau_net, label = 'A -> U; {0}'.format(enzyme_AU))
        ax.legend()
        ax.set_title('Jpx Flux: {0}'.format(jpx_list[-1], fontweight = 'bold'))            
        ax.set_xlabel('Time', fontweight = 'bold')
        ax.set_ylabel('Flux', fontweight = 'bold')
        
    if mode == 'concentration':
        fig, ax = plt.subplots(1, 1, figsize = (12, 5))   
        ax.plot(times_including_initial, S_total_list,label = 'S; {0}'.format(S_total_list[-1]))
        ax.plot(times_including_initial, A_total_list,label = 'A; {0}'.format(A_total_list[-1]))
        ax.plot(times_including_initial, B_total_list,label = 'B; {0}'.format(B_total_list[-1]))  
        ax.plot(times_including_initial, P_total_list,label = 'P; {0}'.format(P_total_list[-1]))
        ax.legend()
        ax.set_title('[P]: {0}'.format(P_total_list[-1], fontweight = 'bold'))
        ax.set_xlabel('Time', fontweight = 'bold')
        ax.set_ylabel('Flux', fontweight = 'bold')

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell defines the functions we need for our interface. 
#----------------------------------------------------------------------------------------------------------------

filename = ''

def initialize_elements_MCA(total_time_slider_value = 10000, timestep_slider_value = 1,                   
                       S_slider_value = 10, A_slider_value = 0, B_slider_value = 0, P_slider_value = 0, LS_slider_value = 0,
                       LA_slider_value = 0, LB_slider_value = 0, LP_slider_value = 0,
                       k_SA_slider_value = 0.1, k_SA_b_slider_value = 0.1, k_AB_slider_value = 0.1,
                       k_AB_b_slider_value = 0.01, k_BP_slider_value = 0.1, k_BP_b_slider_value = 0.01, k_PX_slider_value = 0.1,
                       k_AU_slider_value = 0.1, enzyme_SA_slider_value = 1, enzyme_AB_slider_value = 1, enzyme_BP_slider_value = 1,
                       enzyme_PX_slider_value = 1, enzyme_AU_slider_value = 1, mode_drop_value = 'flux'):

    dictionary_of_elements = {'total_time_slider': widgets.IntSlider(min = 1, max = 1000, step = 1, value = total_time_slider_value),
                              'timestep_slider': widgets.FloatSlider(min = 0.01, max = 1, step = 0.01, value = timestep_slider_value),
                              'S_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = S_slider_value),
                              'A_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = A_slider_value),
                              'B_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = B_slider_value),
                              'P_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = P_slider_value),
                              'LS_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LS_slider_value),
                              'LA_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LA_slider_value),
                              'LB_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LB_slider_value),
                              'LP_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LP_slider_value),
                              'k_SA_slider': widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = k_SA_slider_value),
                              'k_SA_b_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = k_SA_b_slider_value),
                              'k_AB_slider': widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = k_AB_slider_value),
                              'k_AB_b_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = k_AB_b_slider_value),
                              'k_BP_slider': widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = k_BP_slider_value),
                              'k_BP_b_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = k_BP_b_slider_value),
                              'k_AU_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = k_AU_slider_value),
                              'k_PX_slider':widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = k_PX_slider_value),
                              'enzyme_SA_slider':widgets.FloatSlider(min = 0.1, max = 2.0, step = 0.01, value = enzyme_SA_slider_value),
                              'enzyme_AB_slider':widgets.FloatSlider(min = 0.1, max = 2.0, step = 0.01, value = enzyme_AB_slider_value),
                              'enzyme_BP_slider':widgets.FloatSlider(min = 0.1, max = 2.0, step = 0.01, value = enzyme_BP_slider_value),
                              'enzyme_PX_slider':widgets.FloatSlider(min = 0.1, max = 2.0, step = 0.01, value = enzyme_PX_slider_value),
                              'enzyme_AU_slider':widgets.FloatSlider(min = 0.1, max = 2.0, step = 0.01, value = enzyme_AU_slider_value),
                              'mode_drop': widgets.Dropdown(options=[('flux', 'flux'),('concentration', 'concentration'),('no_au', 'no_au')], value = mode_drop_value),
                              'button': widgets.Button(description = 'Run')}

    return dictionary_of_elements


def on_button_clicked(_):                          
    with out:
        clear_output()
        MCA_interactive()
        show_inline_matplotlib_plots()
        
def MCA_interactive():                            
    total_time = elements['total_time_slider'].value
    timestep = elements['timestep_slider'].value
    S = elements['S_slider'].value
    A = elements['A_slider'].value
    B = elements['B_slider'].value
    P = elements['P_slider'].value
    LS = elements['LS_slider'].value
    LA = elements['LA_slider'].value
    LB = elements['LB_slider'].value
    LP = elements['LP_slider'].value
    k_SA = elements['k_SA_slider'].value
    k_SA_b = elements['k_SA_b_slider'].value
    k_AB = elements['k_AB_slider'].value
    k_AB_b = elements['k_AB_b_slider'].value
    k_BP = elements['k_BP_slider'].value
    k_BP_b = elements['k_BP_b_slider'].value
    k_PX = elements['k_PX_slider'].value
    k_AU = elements['k_AU_slider'].value
    enzyme_SA = elements['enzyme_SA_slider'].value
    enzyme_AB = elements['enzyme_AB_slider'].value
    enzyme_BP = elements['enzyme_BP_slider'].value
    enzyme_PX = elements['enzyme_PX_slider'].value
    enzyme_AU = elements['enzyme_AU_slider'].value
    mode = elements['mode_drop'].value

    MCA(total_time, timestep, S, A, B, P, LS, LA, LB,
        LP, k_SA, k_SA_b, k_AB, k_AB_b, k_BP, k_BP_b, k_PX, k_AU,
          enzyme_SA, enzyme_AB, enzyme_BP, enzyme_PX, enzyme_AU, mode)

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell generates the interface that we will use for the following exercise
#----------------------------------------------------------------------------------------------------------------

elements = initialize_elements_MCA(mode_drop_value = 'no_au')   

elements['button'].on_click(on_button_clicked)         
                                                      

out = widgets.Output()                                 

grid = widgets.GridspecLayout(8, 6)                     

grid[0,0] = widgets.HTML('<b>Total Time</b>') 
grid[0,1] = elements['total_time_slider']
grid[0,2] = widgets.HTML('<b>Timestep Size</b>')
grid[0,3] = elements['timestep_slider']
grid[0,4] = widgets.HTML('<b>Constant [S]</b>')
grid[0,5] = elements['S_slider']


grid[1,0] = widgets.HTML('<b>Initial [A]</b>')
grid[1,1] = elements['A_slider']
grid[1,2] = widgets.HTML('<b>Initial [B]</b>')
grid[1,3] = elements['B_slider']
grid[1,4] = widgets.HTML('<b>Initial [P]</b>')
grid[1,5] = elements['P_slider']

grid[2,0] = widgets.HTML('<b>Constant [LS]</b>')
grid[2,1] = elements['LS_slider']
grid[2,2] = widgets.HTML('<b>Initial [LA]</b>')
grid[2,3] = elements['LA_slider']
grid[2,4] = widgets.HTML('<b>Initial [LB]</b>')
grid[2,5] = elements['LB_slider']

grid[3,0] = widgets.HTML('<b>Initial [LP]</b>')
grid[3,1] = elements['LP_slider']
grid[3,2] = widgets.HTML('<b>S -> A Rate Constant</b>')
grid[3,3] = elements['k_SA_slider']
grid[3,4] = widgets.HTML('<b>S -> A Rate Constant</b>')
grid[3,5] = elements['k_SA_b_slider']

grid[4,0] = widgets.HTML('<b>A -> B Rate Constant</b>')
grid[4,1] = elements['k_AB_slider']
grid[4,2] = widgets.HTML('<b>A -> B Rate Constant</b>')
grid[4,3] = elements['k_AB_b_slider']
grid[4,4] = widgets.HTML('<b>B -> P Rate Constant</b>')
grid[4,5] = elements['k_BP_slider']

grid[5,0] = widgets.HTML('<b>P -> B Rate Constant</b>')
grid[5,1] = elements['k_BP_b_slider']
grid[5,2] = widgets.HTML('<b>P -> X Rate Constant</b>')
grid[5,3] = elements['k_PX_slider']
grid[5,4] = widgets.HTML('<b>A -> U Rate Constant</b>')
grid[5,5] = elements['k_AU_slider']

grid[6,0] = widgets.HTML('<b>[SA Enzyme]</b>')
grid[6,1] = elements['enzyme_SA_slider']
grid[6,2] = widgets.HTML('<b>[AB Enzyme]</b>')
grid[6,3] = elements['enzyme_AB_slider']
grid[6,4] = widgets.HTML('<b>[BP Enzyme]</b>')
grid[6,5] = elements['enzyme_BP_slider']

grid[7,2] = elements['button']    

display(grid,out) 

<div class="alert alert-block alert-success">
    <b>Discussion:</b> Do the results make sense? Do the control coefficients all sum up to 1? Once all groups have their answers, let's discuss as a whole class. Do not scroll down until we discuss.
</div>

<br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br><br>

## **(1.2)** Other Stuff is Going On

Some groups probably found that our coefficients aren't all adding up. Why would this be? Well, one possibility is that we've omitted an enzyme that is exerting appreciable control over the flux through this pathway out of our analysis. In fact, the code we ran before was actually simulating the following network:

![alt text](BranchedNetwork.png "Title")

This complicates the linear pathway we had before. In the examples we looked at in Day 2, each enzyme could exert anywhere between 0 and 100% control over the final flux **Jpx**. In this case, we have flux towards another metabolite **U** to worry about. With this in mind, let's do the preceding exercise again, but taking **Jau** into account.

<div class="alert alert-block alert-info">

**Instructions:** Run the cell below. For the set of provided initial conditions, determine the control coefficient for all enzymes as we did before.
</div>

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell generates the interface that we will use for the following exercise
#----------------------------------------------------------------------------------------------------------------

elements = initialize_elements_MCA()                       

elements['button'].on_click(on_button_clicked)             
                                                           

out = widgets.Output()                                     

grid = widgets.GridspecLayout(8, 6)                        

grid[0,0] = widgets.HTML('<b>Total Time</b>') 
grid[0,1] = elements['total_time_slider']
grid[0,2] = widgets.HTML('<b>Timestep Size</b>')
grid[0,3] = elements['timestep_slider']
grid[0,4] = widgets.HTML('<b>Constant [S]</b>')
grid[0,5] = elements['S_slider']


grid[1,0] = widgets.HTML('<b>Initial [A]</b>')
grid[1,1] = elements['A_slider']
grid[1,2] = widgets.HTML('<b>Initial [B]</b>')
grid[1,3] = elements['B_slider']
grid[1,4] = widgets.HTML('<b>Initial [P]</b>')
grid[1,5] = elements['P_slider']

grid[2,0] = widgets.HTML('<b>Constant [LS]</b>')
grid[2,1] = elements['LS_slider']
grid[2,2] = widgets.HTML('<b>Initial [LA]</b>')
grid[2,3] = elements['LA_slider']
grid[2,4] = widgets.HTML('<b>Initial [LB]</b>')
grid[2,5] = elements['LB_slider']

grid[3,0] = widgets.HTML('<b>Initial[LP]</b>')
grid[3,1] = elements['LP_slider']
grid[3,2] = widgets.HTML('<b>S -> A Rate Constant</b>')
grid[3,3] = elements['k_SA_slider']
grid[3,4] = widgets.HTML('<b>S -> A Rate Constant</b>')
grid[3,5] = elements['k_SA_b_slider']

grid[4,0] = widgets.HTML('<b>A -> B Rate Constant</b>')
grid[4,1] = elements['k_AB_slider']
grid[4,2] = widgets.HTML('<b>A -> B Rate Constant</b>')
grid[4,3] = elements['k_AB_b_slider']
grid[4,4] = widgets.HTML('<b>B -> P Rate Constant</b>')
grid[4,5] = elements['k_BP_slider']

grid[5,0] = widgets.HTML('<b>P -> B Rate Constant</b>')
grid[5,1] = elements['k_BP_b_slider']
grid[5,2] = widgets.HTML('<b>P -> X Rate Constant</b>')
grid[5,3] = elements['k_PX_slider']
grid[5,4] = widgets.HTML('<b>A -> U Rate Constant</b>')
grid[5,5] = elements['k_AU_slider']

grid[6,0] = widgets.HTML('<b>[SA Enzyme]</b>')
grid[6,1] = elements['enzyme_SA_slider']
grid[6,2] = widgets.HTML('<b>[AB Enzyme]</b>')
grid[6,3] = elements['enzyme_AB_slider']
grid[6,4] = widgets.HTML('<b>[BP Enzyme]</b>')
grid[6,5] = elements['enzyme_BP_slider']

grid[7,0] = widgets.HTML('<b>[AU Enzyme]</b>')
grid[7,1] = elements['enzyme_AU_slider']
grid[7,2] = elements['button']    

display(grid, out) 

<div class="alert alert-block alert-success">
    <b>Discussion:</b> Do the results make sense? Do the control coefficients all sum up to 1? What's the control coefficient for the new enzyme we introduced? Once all groups have their answers, let's discuss as a whole class.
</div>

# **(2.0)** Michaelis-Menten Kinetics

In the first order kinetics we've been looking at, as you add more and more substrate **S** to a reaction **S &#8594; A**, the rate of the reaction **V** increases linearly with a slope **k**, where **k** is the first order rate constant corresponding to that reaction. Now, enzyme concentrations are not currently an element in our model, but we know that if we have a finite concentration of enzyme, there must be some limit to how fast our reaction can proceed. We refer to the parameter defining the maximum rate of reaction as **Vmax**. As one increases the substrate concentration, the rate of reaction might reach **Vmax** very quickly, very slowly, or anywhere in between. How fast reaction rate increases with substrate addition will be a function of the enzyme's affinity for its substrate. **Km** - a parameter corresponding to the substrate concentration at which **1/2 Vmax** is achieved - encodes information about the enzyme's substrate affinity.

## **(2.1)** Adding Michaelis Menten kinetics to our simulations

The outline of our simulation is still the same as it was before. All that changes now is that we need to update our functions for calculating fluxes to use Michaelis-Menten kinetics instead of first order kinetics. Recall that we have a simple four metabolite network, as shown below:

![alt text](SimpleNetwork.png "Title")

Our fluxes V1-V3 will now be calculated with Michaelis-Menten kinetics; that is:

$$ V_1 = \frac{Vmax_{SA}[S]}{Km_{SA}+[S]} $$

$$ V_2 = \frac{Vmax_{AB}[A]}{Km_{AB}+[A]} $$

$$ V_3 = \frac{Vmax_{BP}[B]}{Km_{BP}+[B]} $$

Note that the other essential equations for our simulation remain unchanged.

### Labeled
$$ \frac{dLA}{dt} = (V1*f_S) - (V2*f_A) $$

$$ \frac{dLB}{dt} = (V2*f_A) - (V3*f_B) $$

$$ \frac{dLP}{dt} = (V3*f_B) $$

### Unlabeled
$$ \frac{dA}{dt} = (V1*(1-f_S)) - (V2*(1-f_A)) $$

$$ \frac{dB}{dt} = (V2*(1-f_A)) - (V3*(1-f_B)) $$

$$ \frac{dP}{dt} = (V3*(1-f_B)) $$

## **(2.2)** Simulating with Michaelis-Menten Kinetics

A qualitative difference between first-order and Michaelis-Menten kinetics is the idea that, given a set concentration of enzyme catalyzing a reaction A, there's a maximum possible flux through A past which addition of more substrate doesn't affect flux. 

<div class="alert alert-block alert-info">

**Instructions:** Run the three cells below. In the interactive cell, try changing Vmax and Km values to find total concentration and flux plots that should be impossible using only first order kinetics.

</div>

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------
# In this cell we are defining functions that we will need during our simulation. Note that we change a number of functions
# from their previous iterations to account for Michaelis-Menten kinetics.
#-----------------------------------------------------------------------------------------------------------------------

def evaluate_Flux_MM(Vmax, Km, S):                                 # Defining a function to evaluate fluxes under
                                                                   # Michaelis-Menten kinetics.
    '''
    DESCRIPTION
    
    Takes a rate constant k and a substrate concentration S 
    and returns a flux value.
    
    INPUTS
    
    Vmax = A numeric value for the Vmax of the reaction
    
    Km = A numeric value for the Km of the reaction
    
    S = A numeric value for the concentration of a substrate S
    
    OUTPUTS
    
    flux = flux, or reaction rate, given substrate 
    concentration and a first order rate constant
    '''
    
    flux = (Vmax * S) / (Km + S)                                   # Defining flux as the product of the provided
                                                                   # Vmax, Km, and substrate concentration 
            
    return flux                                                    


def evaluate_dLAdt(Jsa, Jab, f_s, f_a):                              
    dLAdt = f_s * Jsa - Jab * f_a                                        
    return dLAdt

def evaluate_dLBdt(Jab, Jbp, f_a, f_b):
    dLBdt = f_a * Jab - Jbp * f_b
    return dLBdt

def evaluate_dLPdt(Jbp, f_b):
    dLPdt = Jbp * f_b
    return dLPdt

def evaluate_fractional_labeling(current_labeled, current_total):  
    if current_total == 0:                                        
        f = 0                                                     
    else:                                                         
        f = current_labeled / current_total                       
    return f

#-----------------------------------------------------------------------------------------------------------------------
# Now we can define the function that runs the simulation. The code here is extremely similar to what we saw before for 
# the same model architecture. The only difference is that our fluxes are calculated using Michaelis-Menten kinetics. 
#-----------------------------------------------------------------------------------------------------------------------


def kinetic_simulation_MM(total_time = 50, timestep = 0.01, switch_point = 20, S = 0, A = 0, B = 0, 
        P = 0, LS = 100, LA = 0, LB = 0, LP = 0, S_switch = 0, LS_switch = 0, Vmax_SA = 10, Km_SA = 10,
        Vmax_AB = 10, Km_AB = 10, Vmax_BP = 10, Km_BP = 10, export = False, compare = 'None', filename = '',
        subset = False): 
    
    '''
    DESCRIPTION
    
    A function that carries out a metabolic modeling simulation using first-order kinetics.
    Accepts arguments to both export generated data and import another group's data to 
    compare with simulation results. Can also subset generated data and add random noise
    before exporting.
    
    INPUTS
    
    total_time = A numeric value with arbitrary units that determines how long to run
    the simulation for.
    
    timestep = A numeric value representing how much to increment the time value each
    loop of the simulation.
    
    switch_point = A numeric value representing at which point to switch from an initial
    concentration of labeled and unlabeled S to a "post-switch" value for these same
    quantities
    
    S = A numeric value representing the set concentration of unlabeled S pre-switch
    
    A = A numeric value representing the initial concentration of unlabeled A
    
    B = A numeric value representing the initial concentration of unlabeled B
    
    P = A numeric value representing the initial concentration of unlableed P
    
    LS = A numeric value representing the set concentration of labeled S pre-switch
    
    LA = A numeric value representing the initial concentration of labeled A
    
    LB = A numeric value representing the initial concentration of labeled B
    
    LP = A numeric value representing the initial concentration of labeled P
    
    S_switch = A numeric value representing the set concentration of unlabeled 
    S post-switch
    
    LS_switch = A numeric value representing the set concentration of labeled 
    S post-switch    
    
    Vmax_SA = A numeric value representing the Vmax for the reaction S -> A
    
    Km_SA = A numeric value representing the Km for the reaction S -> A
    
    Vmax_AB = A numeric value representing the Vmax for the reaction S -> A
    
    Km_AB = A numeric value representing the Km for the reaction S -> A
    
    Vmax_BP = A numeric value representing the Vmax for the reaction S -> A
    
    Km_BP = A numeric value representing the Km for the reaction S -> A
    
    export = A boolean value that determines whether or not to export the 
    simulated data as a .csv file
    
    compare = A boolean value that determines whether or not to load in
    another group's data from a .csv file and plot it out
    
    filename = A string corresponding to the name of the file loaded in
    if compare == True
    
    subset = A boolean value that determines whether or not to subset and
    add noise to the simulated data before exporting (only matters if 
    export == True)
    
    OUTPUTS
    
    None
    
    '''
    
    times = np.arange(0, total_time, timestep)      
    
    S_total = S + LS                               
    A_total = A + LA                                
    B_total = B + LB
    P_total = P + LP
    
    S_list = [S]                                                                               
    A_list = [A]                                                                                    
    B_list = [B]                                    
    P_list = [P]                                   
    
    LS_list = [LS]                                 
    LA_list = [LA]
    LB_list = [LB]
    LP_list = [LP]
        
    S_total_list = [S_total]                       
    A_total_list = [A_total]
    B_total_list = [B_total]
    P_total_list = [P_total]

    
    fs_list = [evaluate_fractional_labeling(LS, S_total)] 
    fa_list = [evaluate_fractional_labeling(LA, A_total)] 
    fb_list = [evaluate_fractional_labeling(LB, B_total)] 
    fp_list = [evaluate_fractional_labeling(LP, P_total)]
        
    jsa_list = [evaluate_Flux_MM(Vmax_SA, Km_SA, S_total_list[-1])]     # Here, we generate our initial flux values
    jab_list = [evaluate_Flux_MM(Vmax_AB, Km_AB, A_total_list[-1])]     # using the evaluate_Flux_MM() function
    jbp_list = [evaluate_Flux_MM(Vmax_BP, Km_BP, B_total_list[-1])]    
    
    for i in times:                                  
        
        if i >= switch_point:                        
            current_S = S_switch                    
            current_LS = LS_switch
        else:                                        
            current_S = S
            current_LS = LS
        
        current_Jsa = evaluate_Flux_MM(Vmax_SA, Km_SA, S_total_list[-1]) # Evaluating fluxes using Michaelis-Menten flux function 
        current_Jab = evaluate_Flux_MM(Vmax_AB, Km_AB, A_total_list[-1]) # and Vmax + Km parameters for each reaction
        current_Jbp = evaluate_Flux_MM(Vmax_BP, Km_BP, B_total_list[-1]) 

        current_A = A_list[-1] + (evaluate_dLAdt(current_Jsa, current_Jab, (1 - fs_list[-1]), (1 - fa_list[-1])) * timestep)
        
        current_B = B_list[-1] + (evaluate_dLBdt(current_Jab, current_Jbp, (1 - fa_list[-1]), (1 - fb_list[-1])) * timestep)
        
        current_P = P_list[-1] + (evaluate_dLPdt(current_Jbp, (1 - fb_list[-1])) * timestep)
        
        current_LA = LA_list[-1] + (evaluate_dLAdt(current_Jsa, current_Jab, fs_list[-1], fa_list[-1]) * timestep)
        
        current_LB = LB_list[-1] + (evaluate_dLBdt(current_Jab, current_Jbp, fa_list[-1], fb_list[-1]) * timestep)
        
        current_LP = LP_list[-1] + (evaluate_dLPdt(current_Jbp, fb_list[-1]) * timestep)
        
        current_S_total = current_S + current_LS
        current_A_total = current_A + current_LA
        current_B_total = current_B + current_LB
        current_P_total = current_P + current_LP
        
        current_fs = evaluate_fractional_labeling(current_LS, current_S_total)
        current_fa = evaluate_fractional_labeling(current_LA, current_A_total)
        current_fb = evaluate_fractional_labeling(current_LB, current_B_total)
        current_fp = evaluate_fractional_labeling(current_LP, current_P_total)
        
        LS_list.append(current_LS)                      
        LA_list.append(current_LA)                     
        LB_list.append(current_LB)
        LP_list.append(current_LP)

        S_list.append(current_S)
        A_list.append(current_A)
        B_list.append(current_B)
        P_list.append(current_P)

        S_total_list.append(current_S_total)
        A_total_list.append(current_A_total)
        B_total_list.append(current_B_total)
        P_total_list.append(current_P_total)

        fs_list.append(current_fs)
        fa_list.append(current_fa)
        fb_list.append(current_fb)
        fp_list.append(current_fp)
                
        jsa_list.append(current_Jsa)
        jab_list.append(current_Jab)
        jbp_list.append(current_Jbp) 
    
    times_including_initial = np.insert(times, 0, times[0] - timestep) 

    if compare == 'None':                                           
        
        fig, ax = plt.subplots(1, 3, figsize = (16, 5))             
        ax = ax.flatten()
                
        ax[0].plot(times_including_initial, S_total_list, label = 'S')  
        ax[0].plot(times_including_initial, A_total_list, label = 'A')  
        ax[0].plot(times_including_initial, B_total_list, label = 'B') 
        ax[0].plot(times_including_initial, P_total_list, label = 'P')  
        ax[0].legend()                                                  
        ax[0].set_title('Total Concentration Over Time',                
                        fontweight = 'bold')
        ax[0].set_xlabel('Time', fontweight = 'bold')                   
        ax[0].set_ylabel('Total Concentration', fontweight = 'bold')   

        ax[1].plot(times_including_initial, jsa_list, label = 'S -> A') 
        ax[1].plot(times_including_initial, jab_list, label = 'A -> B') 
        ax[1].plot(times_including_initial, jbp_list, label = 'B -> P')
        ax[1].legend()                                                  
        ax[1].set_title('Fluxes', fontweight = 'bold')                  
        ax[1].set_xlabel('Time', fontweight = 'bold')                   
        ax[1].set_ylabel('Flux', fontweight = 'bold')                  

        ax[2].plot(times_including_initial, fs_list, label = 'S')       
        ax[2].plot(times_including_initial, fa_list, label = 'A')       
        ax[2].plot(times_including_initial, fb_list, label = 'B')      
        ax[2].plot(times_including_initial, fp_list, label = 'P')       
        ax[2].legend()                                                  
        ax[2].set_title('Fraction Labeled Over Time',                   
                        fontweight = 'bold')
        ax[2].set_xlabel('Time', fontweight = 'bold')                   
        ax[2].set_ylabel('Total Concentration', fontweight = 'bold')    
            
    elif compare == 'No Subsetting':                                    
                
        other_data = pd.read_csv(filename)                          

        fig, ax = plt.subplots(2, 3, figsize = (16, 10))           
        ax = ax.flatten()
        
        ax[0].plot(times_including_initial, S_total_list, label = 'S') 
        ax[0].plot(times_including_initial, A_total_list, label = 'A') 
        ax[0].plot(times_including_initial, B_total_list, label = 'B')
        ax[0].plot(times_including_initial, P_total_list, label = 'P') 
        ax[0].legend()
        ax[0].set_title("Concentration Over Time", fontweight = 'bold')
        ax[0].set_xlabel('Time', fontweight = 'bold')
        ax[0].set_ylabel('Total Concentration', fontweight = 'bold')
        
        ax[1].plot(times_including_initial, jsa_list, label = 'S -> A') 
        ax[1].plot(times_including_initial, jab_list, label = 'A -> B') 
        ax[1].plot(times_including_initial, jbp_list, label = 'B -> P')  
        ax[1].legend()
        ax[1].set_title("Fluxes Over Time", fontweight = 'bold')
        ax[1].set_xlabel('Time', fontweight = 'bold')
        ax[1].set_ylabel('Flux', fontweight = 'bold')
        
        ax[2].plot(times_including_initial, fs_list, label = 'S') 
        ax[2].plot(times_including_initial, fa_list, label = 'A') 
        ax[2].plot(times_including_initial, fb_list, label = 'B')
        ax[2].plot(times_including_initial, fp_list, label = 'P')
        ax[2].legend()
        ax[2].set_title("Fraction Labeled Over Time", fontweight = 'bold')
        ax[2].set_xlabel('Time', fontweight = 'bold')
        ax[2].set_ylabel('Total Concentration', fontweight = 'bold')
        
        ax[3].scatter(other_data['Times'], other_data['Total_S'], label = 'S')
        ax[3].scatter(other_data['Times'], other_data['Total_A'], label = 'A') 
        ax[3].scatter(other_data['Times'], other_data['Total_B'], label = 'B') 
        ax[3].scatter(other_data['Times'], other_data['Total_P'], label = 'P')     
        ax[3].legend()
        ax[3].set_title("Other Group's Concentration Over Time", fontweight = 'bold')
        ax[3].set_xlabel('Time', fontweight = 'bold')
        ax[3].set_ylabel('Total Concentration', fontweight = 'bold')
                
        ax[4].scatter(other_data['Times'], other_data['SA_Flux'], label = 'S -> A') 
        ax[4].scatter(other_data['Times'], other_data['AB_Flux'], label = 'A -> B') 
        ax[4].scatter(other_data['Times'], other_data['BP_Flux'], label = 'B -> P')  
        ax[4].legend()
        ax[4].set_title("Other Group's Fluxes Over Time", fontweight = 'bold')
        ax[4].set_xlabel('Time', fontweight = 'bold')
        ax[4].set_ylabel('Flux', fontweight = 'bold')
                
        ax[5].scatter(other_data['Times'], other_data['fs'], label = 'S',alpha = 0.5) 
        ax[5].scatter(other_data['Times'], other_data['fa'], label = 'A',alpha = 0.5) 
        ax[5].scatter(other_data['Times'], other_data['fb'], label = 'B',alpha = 0.5) 
        ax[5].scatter(other_data['Times'], other_data['fp'], label = 'P',alpha = 0.5)     
        ax[5].legend()
        ax[5].set_title("Other Group's Fraction Labeled Over Time", fontweight = 'bold')
        ax[5].set_xlabel('Time', fontweight = 'bold')
        ax[5].set_ylabel('Total Concentration', fontweight = 'bold')

    elif compare == 'With Subsetting':                              
                
        other_data = pd.read_csv(filename)                                                                                      

        fig, ax = plt.subplots(1, 3, figsize=(16, 5))              
        ax = ax.flatten()
        
        ax[0].plot(times_including_initial, S_total_list, label = 'S') 
        ax[0].plot(times_including_initial, A_total_list, label = 'A') 
        ax[0].plot(times_including_initial, B_total_list, label = 'B')
        ax[0].plot(times_including_initial, P_total_list, label = 'P')
        ax[0].scatter(other_data['Times'], other_data['Total_S'], label = 'S')  
        ax[0].scatter(other_data['Times'], other_data['Total_A'], label = 'A') 
        ax[0].scatter(other_data['Times'], other_data['Total_B'], label = 'B') 
        ax[0].scatter(other_data['Times'], other_data['Total_P'], label = 'P')             
        ax[0].legend()
        ax[0].set_title("Concentration Over Time", fontweight = 'bold')
        ax[0].set_xlabel('Time', fontweight = 'bold')
        ax[0].set_ylabel('Total Concentration', fontweight = 'bold')
        
        ax[1].plot(times_including_initial, jsa_list, label = 'S -> A')             
        ax[1].plot(times_including_initial, jab_list, label = 'A -> B')             
        ax[1].plot(times_including_initial, jbp_list, label = 'B -> P')
        ax[1].scatter(other_data['Times'], other_data['SA_Flux'], label = 'S -> A') 
        ax[1].scatter(other_data['Times'], other_data['AB_Flux'], label = 'A -> B') 
        ax[1].scatter(other_data['Times'], other_data['BP_Flux'], label = 'B -> P')          
        ax[1].legend()
        ax[1].set_title("Fluxes Over Time", fontweight = 'bold')
        ax[1].set_xlabel('Time', fontweight = 'bold')
        ax[1].set_ylabel('Flux', fontweight = 'bold')
        
        ax[2].plot(times_including_initial, fs_list, label = 'S') 
        ax[2].plot(times_including_initial, fa_list, label = 'A') 
        ax[2].plot(times_including_initial, fb_list, label = 'B')
        ax[2].plot(times_including_initial, fp_list, label = 'P')
        ax[2].scatter(other_data['Times'], other_data['fs'], label = 'S', alpha = 0.5) 
        ax[2].scatter(other_data['Times'], other_data['fa'], label = 'A', alpha = 0.5) 
        ax[2].scatter(other_data['Times'], other_data['fb'], label = 'B', alpha = 0.5) 
        ax[2].scatter(other_data['Times'], other_data['fp'], label = 'P', alpha = 0.5) 
        ax[2].legend()
        ax[2].set_title("Fraction Labeled Over Time", fontweight = 'bold')
        ax[2].set_xlabel('Time', fontweight = 'bold')
        ax[2].set_ylabel('Total Concentration', fontweight = 'bold')
        
    elif compare == 'Only SA':                                      
                
        other_data = pd.read_csv(filename)                         

        fig, ax = plt.subplots(1, 2, figsize = (10, 5))                
        ax = ax.flatten()
        
        ax[0].plot(times_including_initial, S_total_list, label = 'S') 
        ax[0].plot(times_including_initial, A_total_list,label = 'A') 
        ax[0].scatter(other_data['Times'], other_data['Total_S'], label = 'S') 
        ax[0].scatter(other_data['Times'], other_data['Total_A'], label = 'A')        
        ax[0].legend()
        ax[0].set_title("Concentration Over Time", fontweight = 'bold')
        ax[0].set_xlabel('Time', fontweight = 'bold')
        ax[0].set_ylabel('Total Concentration', fontweight = 'bold')
        
        ax[1].plot(times_including_initial, fs_list, label = 'S') 
        ax[1].plot(times_including_initial, fa_list, label = 'A') 
        ax[1].scatter(other_data['Times'], other_data['fs'], label = 'S', alpha = 0.5) 
        ax[1].scatter(other_data['Times'], other_data['fa'], label = 'A', alpha = 0.5)
        ax[1].legend()
        ax[1].set_title("Fraction Labeled Over Time", fontweight = 'bold')
        ax[1].set_xlabel('Time', fontweight = 'bold')
        ax[1].set_ylabel('Total Concentration', fontweight = 'bold')     

    elif compare == 'Only BP':                                     
                
        other_data = pd.read_csv(filename)                         

        fig, ax = plt.subplots(1, 2, figsize = (10, 5))                 
        ax = ax.flatten()
        
        ax[0].plot(times_including_initial, B_total_list, label = 'B') 
        ax[0].plot(times_including_initial, P_total_list, label = 'P') 
        ax[0].scatter(other_data['Times'], other_data['Total_B'], label = 'B') 
        ax[0].scatter(other_data['Times'], other_data['Total_P'], label = 'P')     
        ax[0].legend()
        ax[0].set_title("Concentration Over Time", fontweight = 'bold')
        ax[0].set_xlabel('Time', fontweight = 'bold')
        ax[0].set_ylabel('Total Concentration', fontweight = 'bold')
        
        ax[1].plot(times_including_initial, fb_list, label = 'B') 
        ax[1].plot(times_including_initial, fp_list, label = 'P') 
        ax[1].scatter(other_data['Times'], other_data['fb'], label = 'B', alpha = 0.5) 
        ax[1].scatter(other_data['Times'], other_data['fp'], label = 'P', alpha = 0.5) 
        ax[1].legend()
        ax[1].set_title("Fraction Labeled Over Time", fontweight = 'bold')
        ax[1].set_xlabel('Time', fontweight = 'bold')
        ax[1].set_ylabel('Total Concentration', fontweight = 'bold')         
    
    if export:                                                            
 
        if not subset:                                                   
            
            results_dictionary = {'Times':times_including_initial,        
                                  'Total_S':S_total_list,                 
                                  'Total_A':A_total_list,
                                  'Total_B':B_total_list,
                                  'Total_P':P_total_list,
                                  'SA_Flux':jsa_list,
                                  'AB_Flux':jab_list,
                                  'BP_Flux':jbp_list,
                                  'fs':fs_list,
                                  'fa':fa_list,
                                  'fb':fb_list,
                                  'fp':fp_list}
            results_df = pd.DataFrame(results_dictionary)                 
            results_df.to_csv('GroupX_MMData.csv')                
        
        elif subset:                                                     
                        
            times_numpy = np.array(times_including_initial)[0::100]       
            S_total_numpy = np.array(S_total_list)[0::100]                
            A_total_numpy = np.array(A_total_list)[0::100]                
            B_total_numpy = np.array(B_total_list)[0::100]
            P_total_numpy = np.array(P_total_list)[0::100]
            jsa_numpy = np.array(jsa_list)[0::100]
            jab_numpy = np.array(jab_list)[0::100]
            jbp_numpy = np.array(jbp_list)[0::100]
            fs_numpy = np.array(fs_list)[0::100]
            fa_numpy = np.array(fa_list)[0::100]
            fb_numpy = np.array(fb_list)[0::100]
            fp_numpy = np.array(fp_list)[0::100]

            S_total_noise = np.random.normal(0, 1, size = S_total_numpy.shape) 
            S_total_noise = S_total_noise * 0.1 * S_total_numpy               
            S_noisy_array = S_total_numpy + S_total_noise                      

            A_total_noise = np.random.normal(0, 1, size = A_total_numpy.shape) 
            A_total_noise = A_total_noise * 0.1 * A_total_numpy
            A_noisy_array = A_total_numpy + A_total_noise

            B_total_noise = np.random.normal(0, 1, size = B_total_numpy.shape) 
            B_total_noise = B_total_noise * 0.1 * B_total_numpy
            B_noisy_array = B_total_numpy + B_total_noise

            P_total_noise = np.random.normal(0, 1, size = P_total_numpy.shape) 
            P_total_noise = P_total_noise * 0.1 * P_total_numpy
            P_noisy_array = P_total_numpy + P_total_noise

            jsa_noise = np.random.normal(0, 1, size = jsa_numpy.shape)      
            jsa_noise = jsa_noise * 0.02 * jsa_numpy                       
            jsa_noisy_array = jsa_numpy + jsa_noise
            
            jab_noise = np.random.normal(0, 1, size = jab_numpy.shape)        
            jab_noise = jab_noise * 0.02 * jab_numpy
            jab_noisy_array = jab_numpy + jab_noise
            
            jbp_noise = np.random.normal(0, 1, size = jbp_numpy.shape)       
            jbp_noise = jbp_noise * 0.02 * jbp_numpy
            jbp_noisy_array = jbp_numpy + jbp_noise
            
            fs_noise = np.random.normal(0, 1, size = fs_numpy.shape)         
            fs_noise = fs_noise * 0.02 * fs_numpy                             
            fs_noisy_array = fs_numpy + fs_noise

            fa_noise = np.random.normal(0, 1, size = fa_numpy.shape)          
            fa_noise = fa_noise * 0.02 * fa_numpy
            fa_noisy_array = fa_numpy + fa_noise

            fb_noise = np.random.normal(0, 1, size = fb_numpy.shape)          
            fb_noise = fb_noise * 0.02 * fb_numpy
            fb_noisy_array = fb_numpy + fb_noise

            fp_noise = np.random.normal(0, 1, size = fp_numpy.shape)          
            fp_noise = fp_noise * 0.02 * fp_numpy
            fp_noisy_array = fp_numpy + fp_noise
            
            results_dictionary = {'Times':times_numpy,                   
                                  'Total_S':S_noisy_array,               
                                  'Total_A':A_noisy_array,
                                  'Total_B':B_noisy_array,
                                  'Total_P':P_noisy_array,
                                  'SA_Flux':jsa_noisy_array,
                                  'AB_Flux':jab_noisy_array,
                                  'BP_Flux':jbp_noisy_array,
                                  'fs':fs_noisy_array,
                                  'fa':fa_noisy_array,
                                  'fb':fb_noisy_array,
                                  'fp':fp_noisy_array}
            results_df = pd.DataFrame(results_dictionary)
            results_df.to_csv('GroupX_MMData_Subsampled.csv')
    
    return fig, ax

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell defines the functions we need for our interface.
#----------------------------------------------------------------------------------------------------------------

filename = ''

def initialize_elements_MM(total_time_slider_value = 100, timestep_slider_value = 0.1,                   
                       switch_point_slider_value = 20, S_slider_value = 0, A_slider_value = 0,
                       B_slider_value = 0, P_slider_value = 0, LS_slider_value = 100,
                       LA_slider_value = 0, LB_slider_value = 0, LP_slider_value = 0,
                       S_switch_slider_value = 0, LS_switch_slider_value = 0,
                       Vmax_SA_slider_value = 10, Km_SA_slider_value = 10, Vmax_AB_slider_value = 10, Km_AB_slider_value = 10,
                       Vmax_BP_slider_value = 10, Km_BP_slider_value = 10, compare_drop_value = 'None', subsample_drop_value = False,
                       export_drop_value = False):

    dictionary_of_elements = {'total_time_slider': widgets.IntSlider(min = 1, max = 1000, step = 1, value = total_time_slider_value),
                              'timestep_slider': widgets.FloatSlider(min = 0.01, max = 1, step = 0.01, value = timestep_slider_value),
                              'switch_point_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = switch_point_slider_value),
                              'S_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = S_slider_value),
                              'A_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = A_slider_value),
                              'B_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = B_slider_value),
                              'P_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = P_slider_value),
                              'LS_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LS_slider_value),
                              'LA_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LA_slider_value),
                              'LB_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LB_slider_value),
                              'LP_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LP_slider_value),
                              'S_switch_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = S_switch_slider_value),
                              'LS_switch_slider': widgets.IntSlider(min = 0, max = 100 ,step = 1, value = LS_switch_slider_value),
                              'Vmax_SA_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Vmax_SA_slider_value),
                              'Vmax_AB_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Vmax_AB_slider_value),
                              'Vmax_BP_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Vmax_BP_slider_value),
                              'Km_SA_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Km_SA_slider_value),
                              'Km_AB_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Km_AB_slider_value),
                              'Km_BP_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Km_BP_slider_value),
                              'compare_drop': widgets.Dropdown(options = [('None', 'None'), ('No Subsetting', 'No Subsetting'),
                                                                          ('With Subsetting', 'With Subsetting'),('Only SA', 'Only SA'),
                                                                          ('Only BP', 'Only BP')], value = compare_drop_value),
                              'subsample_drop': widgets.Dropdown(options = [('Yes', True),('No', False)], value = subsample_drop_value),
                              'export_drop': widgets.Dropdown(options = [('Yes', True),('No', False)], value = export_drop_value),
                              'button': widgets.Button(description = 'Run')}
    return dictionary_of_elements

def on_button_clicked(_):                           
    with out:
        clear_output()
        kinetic_simulation_MM_interactive(filename)
        show_inline_matplotlib_plots()
        
def kinetic_simulation_MM_interactive(filename):                                     
    total_time = elements['total_time_slider'].value
    timestep = elements['timestep_slider'].value
    switch_point = elements['switch_point_slider'].value
    S = elements['S_slider'].value
    A = elements['A_slider'].value
    B = elements['B_slider'].value
    P = elements['P_slider'].value
    LS = elements['LS_slider'].value
    LA = elements['LA_slider'].value
    LB = elements['LB_slider'].value
    LP = elements['LP_slider'].value
    S_switch = elements['S_switch_slider'].value
    LS_switch = elements['LS_switch_slider'].value
    Vmax_SA = elements['Vmax_SA_slider'].value
    Vmax_AB = elements['Vmax_AB_slider'].value
    Vmax_BP = elements['Vmax_BP_slider'].value
    Km_SA = elements['Km_SA_slider'].value
    Km_AB = elements['Km_AB_slider'].value
    Km_BP = elements['Km_BP_slider'].value
    compare = elements['compare_drop'].value
    subsample = elements['subsample_drop'].value
    export = elements['export_drop'].value
    
    kinetic_simulation_MM(total_time, timestep, switch_point, S, A, B, P, LS, LA, LB,
        LP, S_switch, LS_switch, Vmax_SA, Km_SA, Vmax_AB, Km_AB,
           Vmax_BP, Km_BP, export, compare, filename, subsample)

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell actually generates the interface that we will use for the following exercise
#----------------------------------------------------------------------------------------------------------------

elements = initialize_elements_MM()                    

elements['button'].on_click(on_button_clicked)         

out = widgets.Output()                                 

grid = widgets.GridspecLayout(8, 6)                    

grid[0,0] = widgets.HTML('<b>Total Time</b>')
grid[0,1] = elements['total_time_slider']
grid[0,2] = widgets.HTML('<b>Timestep Size</b>')
grid[0,3] = elements['timestep_slider']
grid[0,4] = widgets.HTML('<b>Switch Point</b>')
grid[0,5] = elements['switch_point_slider']

grid[1,0] = widgets.HTML('<b>Pre-Switch Constant[S]</b>')
grid[1,1] = elements['S_slider']
grid[1,2] = widgets.HTML('<b>Initial [A]</b>')
grid[1,3] = elements['A_slider']
grid[1,4] = widgets.HTML('<b>Initial [B]</b>')
grid[1,5] = elements['B_slider']

grid[2,0] = widgets.HTML('<b>Initial [P]</b>')
grid[2,1] = elements['P_slider']
grid[2,2] = widgets.HTML('<b>Pre-Switch Constant [LS]</b>')
grid[2,3] = elements['LS_slider']
grid[2,4] = widgets.HTML('<b>Initial [LA]</b>')
grid[2,5] = elements['LA_slider']

grid[3,0] = widgets.HTML('<b>Initial [LB]</b>')
grid[3,1] = elements['LB_slider']
grid[3,2] = widgets.HTML('<b>Initial [LP]</b>')
grid[3,3] = elements['LP_slider']
grid[3,4] = widgets.HTML('<b>Post-Switch Constant [S]</b>')
grid[3,5] = elements['S_switch_slider']

grid[4,0] = widgets.HTML('<b>Post-Switch Constant [LS]</b>')
grid[4,1] = elements['LS_switch_slider']
grid[4,2] = widgets.HTML('<b>S -> A Vmax</b>')
grid[4,3] = elements['Vmax_SA_slider']
grid[4,4] = widgets.HTML('<b>S -> A Km</b>')
grid[4,5] = elements['Km_SA_slider']

grid[5,0] = widgets.HTML('<b>A -> B Vmax</b>')
grid[5,1] = elements['Vmax_AB_slider']
grid[5,2] = widgets.HTML('<b>A -> B Km</b>')
grid[5,3] = elements['Km_AB_slider']
grid[5,4] = widgets.HTML('<b>B -> P Vmax</b>')
grid[5,5] = elements['Vmax_BP_slider']

grid[6,0] = widgets.HTML('<b>B -> P Km</b>')
grid[6,1] = elements['Km_BP_slider']
grid[6,2] = widgets.HTML('<b>Export?</b>')
grid[6,3] = elements['export_drop']
grid[6,4] = widgets.HTML('<b>Subsample?</b>')
grid[6,5] = elements['subsample_drop']

grid[7,0] = widgets.HTML('<b>Comparison</b>')
grid[7,1] = elements['compare_drop']
grid[7,2] = elements['button']    

display(grid,out)

# **(3.0)** Fitting a Complex Reality with a Simple Model

Our attempts to model systems are limited by the quality and resolution of the data we can gather. There may be cases where our data on a system cannot clearly distinguish between first order and Michaelis-Menten kinetics. Let's see an example of this ...

<div class="alert alert-block alert-info">

**Instructions:** Run the four cells below. Take some time and see if you can get a decent fit to the provided dataset using first-order kinetic parameters. Don't worry if the fit isn't perfect; we just want you to give it your best shot. Once you have fits that look reasonable enough, record your parameter values.

</div>

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------
# In this cell we are defining functions that we will need during our simulation, as well as the function that carries
# out the simulation itself. This code defines the first-order kinetic simulation functions - for detailed comments, see
# the Day01 notebook.
#-----------------------------------------------------------------------------------------------------------------------

def evaluate_Flux(k, S):                                           
    flux = k * S                                                                                                                       
    return flux                                                    

def evaluate_dLAdt(Jsa, Jab, f_s, f_a):                              
    dLAdt = f_s * Jsa - Jab * f_a                                                                                          
    return dLAdt

def evaluate_dLBdt(Jab, Jbp, f_a, f_b):
    dLBdt = f_a * Jab - Jbp * f_b
    return dLBdt

def evaluate_dLPdt(Jbp, f_b):
    dLPdt = Jbp * f_b
    return dLPdt

def evaluate_fractional_labeling(current_labeled, current_total):  
    if current_total == 0:                                       
        f = 0                                                    
    else:                                                        
        f = current_labeled / current_total                       
    return f

def kinetic_simulation(total_time = 50, timestep = 0.01, switch_point = 20, S = 0, A = 0, B = 0,  
        P = 0, LS = 100, LA = 0, LB = 0, LP = 0, S_switch = 0, LS_switch = 0, k_SA = 10,
        k_AB = 10, k_BP = 10, export = False, compare = 'None', filename = '',
        subset = False): 
    
    times = np.arange(0, total_time, timestep)      
    
    S_total = S + LS                                 
    A_total = A + LA                                  
    B_total = B + LB
    P_total = P + LP
    
    S_list = [S]                                                                                      
    A_list = [A]                                                                                    
    B_list = [B]                                      
    P_list = [P]                                                                                          
    
    LS_list = [LS]                                    
    LA_list = [LA]
    LB_list = [LB]
    LP_list = [LP]
        
    S_total_list = [S_total]                       
    A_total_list = [A_total]
    B_total_list = [B_total]
    P_total_list = [P_total]

    
    fs_list = [evaluate_fractional_labeling(LS, S_total)] 
    fa_list = [evaluate_fractional_labeling(LA, A_total)] 
    fb_list = [evaluate_fractional_labeling(LB, B_total)] 
    fp_list = [evaluate_fractional_labeling(LP, P_total)]
        
    jsa_list = [evaluate_Flux(k_SA, S_total_list[-1])] 
    jab_list = [evaluate_Flux(k_AB, A_total_list[-1])]
    jbp_list = [evaluate_Flux(k_BP, B_total_list[-1])]

    for i in times:                                 
        
        if i >= switch_point:                       
            current_S = S_switch                    
            current_LS = LS_switch
        else:                                       
            current_S = S
            current_LS = LS
                
        current_Jsa = evaluate_Flux(k_SA, S_total_list[-1]) 
        current_Jab = evaluate_Flux(k_AB, A_total_list[-1])
        current_Jbp = evaluate_Flux(k_BP, B_total_list[-1]) 
        
        current_A = A_list[-1] + (evaluate_dLAdt(current_Jsa, current_Jab, (1 - fs_list[-1]), (1 - fa_list[-1])) * timestep)
        current_B = B_list[-1] + (evaluate_dLBdt(current_Jab, current_Jbp, (1 - fa_list[-1]), (1 - fb_list[-1])) * timestep)
        current_P = P_list[-1] + (evaluate_dLPdt(current_Jbp, (1 - fb_list[-1])) * timestep)
        
        current_LA = LA_list[-1] + (evaluate_dLAdt(current_Jsa, current_Jab, fs_list[-1], fa_list[-1]) * timestep)
        current_LB = LB_list[-1] + (evaluate_dLBdt(current_Jab, current_Jbp, fa_list[-1], fb_list[-1]) * timestep)
        current_LP = LP_list[-1] + (evaluate_dLPdt(current_Jbp, fb_list[-1]) * timestep)
        
        current_S_total = current_S + current_LS
        current_A_total = current_A + current_LA
        current_B_total = current_B + current_LB
        current_P_total = current_P + current_LP
        
        current_fs = evaluate_fractional_labeling(current_LS, current_S_total)
        current_fa = evaluate_fractional_labeling(current_LA, current_A_total)
        current_fb = evaluate_fractional_labeling(current_LB, current_B_total)
        current_fp = evaluate_fractional_labeling(current_LP, current_P_total)
        
        LS_list.append(current_LS)                     
        LA_list.append(current_LA)                     
        LB_list.append(current_LB)
        LP_list.append(current_LP)

        S_list.append(current_S)
        A_list.append(current_A)
        B_list.append(current_B)
        P_list.append(current_P)

        S_total_list.append(current_S_total)
        A_total_list.append(current_A_total)
        B_total_list.append(current_B_total)
        P_total_list.append(current_P_total)

        fs_list.append(current_fs)
        fa_list.append(current_fa)
        fb_list.append(current_fb)
        fp_list.append(current_fp)
                
        jsa_list.append(current_Jsa)
        jab_list.append(current_Jab)
        jbp_list.append(current_Jbp)
    
   
    
    times_including_initial = np.insert(times, 0, times[0] - timestep) 

    if compare == 'None':                                           
        
        fig, ax = plt.subplots(1, 3, figsize = (16, 5))            
        ax = ax.flatten()
                
        ax[0].plot(times_including_initial, S_total_list, label='S') 
        ax[0].plot(times_including_initial, A_total_list, label='A')  
        ax[0].plot(times_including_initial, B_total_list, label='B')  
        ax[0].plot(times_including_initial, P_total_list, label='P')  
        ax[0].legend()                                                
        ax[0].set_title('Total Concentration Over Time',              
                        fontweight = 'bold')
        ax[0].set_xlabel('Time', fontweight = 'bold')                
        ax[0].set_ylabel('Total Concentration', fontweight = 'bold') 

        ax[1].plot(times_including_initial, jsa_list, label = 'S -> A') 
        ax[1].plot(times_including_initial, jab_list, label = 'A -> B') 
        ax[1].plot(times_including_initial, jbp_list, label = 'B -> P') 
        ax[1].legend()                                                  
        ax[1].set_title('Fluxes', fontweight = 'bold')                 
        ax[1].set_xlabel('Time', fontweight = 'bold')                  
        ax[1].set_ylabel('Flux', fontweight = 'bold')                  

        ax[2].plot(times_including_initial, fs_list, label = 'S')     
        ax[2].plot(times_including_initial, fa_list, label = 'A')       
        ax[2].plot(times_including_initial, fb_list, label = 'B')    
        ax[2].plot(times_including_initial, fp_list, label = 'P')       
        ax[2].legend()                                                
        ax[2].set_title('Fraction Labeled Over Time',                 
                        fontweight = 'bold')
        ax[2].set_xlabel('Time', fontweight = 'bold')                  
        ax[2].set_ylabel('Total Concentration', fontweight = 'bold')   
            
    elif compare == 'No Subsetting':                               
                
        other_data = pd.read_csv(filename)                          

        fig, ax = plt.subplots(1, 2, figsize = (10, 5))           
        ax = ax.flatten()
        
        ax[0].plot(times_including_initial, S_total_list, label = 'S') 
        ax[0].plot(times_including_initial, A_total_list, label = 'A') 
        ax[0].plot(times_including_initial, B_total_list, label = 'B')
        ax[0].plot(times_including_initial, P_total_list, label = 'P') 
        ax[0].legend()
        ax[0].set_title("Concentration Over Time", fontweight = 'bold')
        ax[0].set_xlabel('Time', fontweight = 'bold')
        ax[0].set_ylabel('Total Concentration',fontweight = 'bold')
        
        ax[1].plot(other_data['Times'], other_data['Total_S'], label = 'S') 
        ax[1].plot(other_data['Times'], other_data['Total_A'], label = 'A') 
        ax[1].plot(other_data['Times'], other_data['Total_B'], label = 'B')
        ax[1].plot(other_data['Times'], other_data['Total_P'], label = 'P')     
        ax[1].legend()
        ax[1].set_title("Other Group's Concentration Over Time", fontweight = 'bold')
        ax[1].set_xlabel('Time', fontweight = 'bold')
        ax[1].set_ylabel('Total Concentration', fontweight = 'bold')

    elif compare == 'With Subsetting':                                                
                
        other_data = pd.read_csv(filename)                         

        fig, ax = plt.subplots(1, 1, figsize=(5, 5))              
        
        ax.plot(times_including_initial, S_total_list, label = 'S') 
        ax.plot(times_including_initial, A_total_list, label = 'A') 
        ax.plot(times_including_initial, B_total_list, label = 'B')
        ax.plot(times_including_initial, P_total_list, label = 'P')
        ax.scatter(other_data['Times'], other_data['Total_S'], label = 'S')
        ax.scatter(other_data['Times'], other_data['Total_A'], label = 'A') 
        ax.scatter(other_data['Times'], other_data['Total_B'], label = 'B') 
        ax.scatter(other_data['Times'], other_data['Total_P'], label = 'P')             
        ax.legend()
        ax.set_title("Concentration Over Time", fontweight = 'bold')
        ax.set_xlabel('Time', fontweight = 'bold')
        ax.set_ylabel('Total Concentration', fontweight = 'bold')

    elif compare == 'Only SA':                                      
                
        other_data = pd.read_csv(filename)                         

        fig, ax = plt.subplots(1, 2, figsize = (10, 5))             
        ax = ax.flatten()
        
        ax[0].plot(times_including_initial, S_total_list, label = 'S')         
        ax[0].plot(times_including_initial, A_total_list, label = 'A')         
        ax[0].scatter(other_data['Times'], other_data['Total_S'], label = 'S') 
        ax[0].scatter(other_data['Times'], other_data['Total_A'], label = 'A')    
        ax[0].legend()
        ax[0].set_title("Concentration Over Time", fontweight = 'bold')
        ax[0].set_xlabel('Time', fontweight = 'bold')
        ax[0].set_ylabel('Total Concentration', fontweight = 'bold')
        
        ax[1].plot(times_including_initial, fs_list, label = 'S')                      
        ax[1].plot(times_including_initial, fa_list, label = 'A')                      
        ax[1].scatter(other_data['Times'], other_data['fs'], label = 'S', alpha = 0.5) 
        ax[1].scatter(other_data['Times'], other_data['fa'], label = 'A', alpha = 0.5) 
        ax[1].legend()
        ax[1].set_title("Fraction Labeled Over Time", fontweight = 'bold')
        ax[1].set_xlabel('Time', fontweight = 'bold')
        ax[1].set_ylabel('Total Concentration', fontweight = 'bold')     

    elif compare == 'Only BP':                                     
                
        other_data = pd.read_csv(filename)                          

        fig, ax = plt.subplots(1, 2, figsize = (10, 5))            
        ax = ax.flatten()
        
        ax[0].plot(times_including_initial, B_total_list, label = 'B')         
        ax[0].plot(times_including_initial, P_total_list, label = 'P')         
        ax[0].scatter(other_data['Times'], other_data['Total_B'], label = 'B') 
        ax[0].scatter(other_data['Times'], other_data['Total_P'], label = 'P')    
        ax[0].legend()
        ax[0].set_title("Concentration Over Time", fontweight = 'bold')
        ax[0].set_xlabel('Time', fontweight = 'bold')
        ax[0].set_ylabel('Total Concentration', fontweight = 'bold')
        
        ax[1].plot(times_including_initial, fb_list, label = 'B')                     
        ax[1].plot(times_including_initial, fp_list, label = 'P')                      
        ax[1].scatter(other_data['Times'], other_data['fb'], label = 'B', alpha = 0.5) 
        ax[1].scatter(other_data['Times'], other_data['fp'], label = 'P', alpha = 0.5) 
        ax[1].legend()
        ax[1].set_title("Fraction Labeled Over Time", fontweight = 'bold')
        ax[1].set_xlabel('Time', fontweight = 'bold')
        ax[1].set_ylabel('Total Concentration', fontweight = 'bold')         
    
    if export:                                                            
        if not subset:                                                   
            results_dictionary = {'Times':times_including_initial,        
                                  'Total_S':S_total_list,                 
                                  'Total_A':A_total_list,
                                  'Total_B':B_total_list,
                                  'Total_P':P_total_list,
                                  'SA_Flux':jsa_list,
                                  'AB_Flux':jab_list,
                                  'BP_Flux':jbp_list,
                                  'fs':fs_list,
                                  'fa':fa_list,
                                  'fb':fb_list,
                                  'fp':fp_list}
            results_df = pd.DataFrame(results_dictionary)                
            results_df.to_csv('GroupX_FirstOrderData.csv')                
        
        elif subset:                                                                  
            times_numpy = np.array(times_including_initial)[0::100]       
            S_total_numpy = np.array(S_total_list)[0::100]                
            A_total_numpy = np.array(A_total_list)[0::100]                
            B_total_numpy = np.array(B_total_list)[0::100]
            P_total_numpy = np.array(P_total_list)[0::100]
            jsa_numpy = np.array(jsa_list)[0::100]
            jab_numpy = np.array(jab_list)[0::100]
            jbp_numpy = np.array(jbp_list)[0::100]
            fs_numpy = np.array(fs_list)[0::100]
            fa_numpy = np.array(fa_list)[0::100]
            fb_numpy = np.array(fb_list)[0::100]
            fp_numpy = np.array(fp_list)[0::100]
                
            S_total_noise = np.random.normal(0, 1, size = S_total_numpy.shape)
            S_total_noise = S_total_noise * 0.1 * S_total_numpy             
            S_noisy_array = S_total_numpy + S_total_noise                                                                           

            A_total_noise = np.random.normal(0, 1, size = A_total_numpy.shape)        
            A_total_noise = A_total_noise * 0.1 * A_total_numpy
            A_noisy_array = A_total_numpy + A_total_noise

            B_total_noise = np.random.normal(0, 1, size = B_total_numpy.shape)        
            B_total_noise = B_total_noise * 0.1 * B_total_numpy
            B_noisy_array = B_total_numpy + B_total_noise

            P_total_noise = np.random.normal(0, 1, size = P_total_numpy.shape)        
            P_total_noise = P_total_noise * 0.1 * P_total_numpy
            P_noisy_array = P_total_numpy + P_total_noise

            jsa_noise = np.random.normal(0, 1, size = jsa_numpy.shape)    
            jsa_noise = jsa_noise * 0.02 * jsa_numpy                     
            jsa_noisy_array = jsa_numpy + jsa_noise
            
            jab_noise = np.random.normal(0, 1, size = jab_numpy.shape)   
            jab_noise = jab_noise * 0.02 * jab_numpy
            jab_noisy_array = jab_numpy + jab_noise
            
            jbp_noise = np.random.normal(0, 1, size = jbp_numpy.shape)    
            jbp_noise = jbp_noise * 0.02 * jbp_numpy
            jbp_noisy_array = jbp_numpy + jbp_noise
            
            fs_noise = np.random.normal(0, 1, size = fs_numpy.shape)     
            fs_noise = fs_noise * 0.02 * fs_numpy                        
            fs_noisy_array = fs_numpy + fs_noise

            fa_noise = np.random.normal(0, 1, size = fa_numpy.shape)     
            fa_noise = fa_noise * 0.02 * fa_numpy
            fa_noisy_array = fa_numpy + fa_noise

            fb_noise = np.random.normal(0, 1, size = fb_numpy.shape)      
            fb_noise = fb_noise * 0.02 * fb_numpy
            fb_noisy_array = fb_numpy + fb_noise

            fp_noise = np.random.normal(0, 1, size = fp_numpy.shape)     
            fp_noise = fp_noise * 0.02 * fp_numpy
            fp_noisy_array = fp_numpy + fp_noise
            
            results_dictionary = {'Times':times_numpy,                   
                                  'Total_S':S_noisy_array,               
                                  'Total_A':A_noisy_array,
                                  'Total_B':B_noisy_array,
                                  'Total_P':P_noisy_array,
                                  'SA_Flux':jsa_noisy_array,
                                  'AB_Flux':jab_noisy_array,
                                  'BP_Flux':jbp_noisy_array,
                                  'fs':fs_noisy_array,
                                  'fa':fa_noisy_array,
                                  'fb':fb_noisy_array,
                                  'fp':fp_noisy_array}
            results_df = pd.DataFrame(results_dictionary)
            results_df.to_csv('GroupX_FirstOrderData_Subsampled.csv')
    
    return fig, ax

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell defines the functions we need for our interface. 
#----------------------------------------------------------------------------------------------------------------

def initialize_elements(total_time_slider_value = 100, timestep_slider_value = 0.1,                   
                       switch_point_slider_value = 20, S_slider_value = 0, A_slider_value = 0,
                       B_slider_value = 0, P_slider_value = 0, LS_slider_value = 100,
                       LA_slider_value = 0, LB_slider_value = 0, LP_slider_value = 0,
                       S_switch_slider_value = 0, LS_switch_slider_value = 0,
                       k_SA_slider_value = 0.1, k_AB_slider_value = 0.1,
                       k_BP_slider_value = 0.1, compare_drop_value = 'None', subsample_drop_value = False,
                       export_drop_value = False):

    dictionary_of_elements = {'total_time_slider': widgets.IntSlider(min = 1, max = 1000, step = 1, value = total_time_slider_value),
                              'timestep_slider': widgets.FloatSlider(min = 0.01, max = 1, step = 0.01, value = timestep_slider_value),
                              'switch_point_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = switch_point_slider_value),
                              'S_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = S_slider_value),
                              'A_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = A_slider_value),
                              'B_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = B_slider_value),
                              'P_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = P_slider_value),
                              'LS_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LS_slider_value),
                              'LA_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LA_slider_value),
                              'LB_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LB_slider_value),
                              'LP_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LP_slider_value),
                              'S_switch_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = S_switch_slider_value),
                              'LS_switch_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LS_switch_slider_value),
                              'k_SA_slider': widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = k_SA_slider_value),
                              'k_AB_slider': widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = k_AB_slider_value),
                              'k_BP_slider': widgets.FloatSlider(min = 0, max = 1, step = 0.01, value = k_BP_slider_value),
                              'compare_drop': widgets.Dropdown(options = [('None', 'None'), ('No Subsetting', 'No Subsetting'),
                                                                        ('With Subsetting', 'With Subsetting'),
                                                                        ('Only SA', 'Only SA'), ('Only BP', 'Only BP')], value = compare_drop_value),
                              'subsample_drop': widgets.Dropdown(options = [('Yes', True), ('No', False)], value = subsample_drop_value),
                              'export_drop': widgets.Dropdown(options = [('Yes', True),('No', False)], value = export_drop_value),
                              'button': widgets.Button(description = 'Run')}
    return dictionary_of_elements

def on_button_clicked(_):                           
    with out:
        clear_output()
        kinetic_simulation_interactive(filename)
        show_inline_matplotlib_plots()
        
def kinetic_simulation_interactive(filename):                                     
    total_time = elements['total_time_slider'].value
    timestep = elements['timestep_slider'].value
    switch_point = elements['switch_point_slider'].value
    S = elements['S_slider'].value
    A = elements['A_slider'].value
    B = elements['B_slider'].value
    P = elements['P_slider'].value
    LS = elements['LS_slider'].value
    LA = elements['LA_slider'].value
    LB = elements['LB_slider'].value
    LP = elements['LP_slider'].value
    S_switch = elements['S_switch_slider'].value
    LS_switch = elements['LS_switch_slider'].value
    k_SA = elements['k_SA_slider'].value
    k_AB = elements['k_AB_slider'].value
    k_BP = elements['k_BP_slider'].value
    compare = elements['compare_drop'].value
    subsample = elements['subsample_drop'].value
    export = elements['export_drop'].value
    
    kinetic_simulation(total_time, timestep, switch_point, S, A, B, P, LS, LA, LB,
        LP, S_switch, LS_switch, k_SA, k_AB, k_BP, export, compare,
        filename, subsample)

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This cell loads in the file we'll be using
#----------------------------------------------------------------------------------------------------------------

filename = 'Day03_Exercise_FullData.csv'

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell actually generates the interface that we will use for the following exercise
#----------------------------------------------------------------------------------------------------------------

elements = initialize_elements(compare_drop_value = 'No Subsetting')    

elements['button'].on_click(on_button_clicked)         

out = widgets.Output()                                 

grid = widgets.GridspecLayout(6, 6)                    

grid[0,0] = widgets.HTML('<b>Total Time</b>')
grid[0,1] = elements['total_time_slider']
grid[0,2] = widgets.HTML('<b>Timestep Size</b>')
grid[0,3] = elements['timestep_slider']
grid[0,4] = widgets.HTML('<b>Switch Point</b>')
grid[0,5] = elements['switch_point_slider']

grid[1,0] = widgets.HTML('<b>Pre-Switch Constant [S]</b>')
grid[1,1] = elements['S_slider']
grid[1,2] = widgets.HTML('<b>Initial [A]</b>')
grid[1,3] = elements['A_slider']
grid[1,4] = widgets.HTML('<b>Initial [B]</b>')
grid[1,5] = elements['B_slider']

grid[2,0] = widgets.HTML('<b>Initial [P]</b>')
grid[2,1] = elements['P_slider']
grid[2,2] = widgets.HTML('<b>Pre-Switch Constant [LS]</b>')
grid[2,3] = elements['LS_slider']
grid[2,4] = widgets.HTML('<b>Initial [LA]</b>')
grid[2,5] = elements['LA_slider']

grid[3,0] = widgets.HTML('<b>Initial [LB]</b>')
grid[3,1] = elements['LB_slider']
grid[3,2] = widgets.HTML('<b>Initial [LP]</b>')
grid[3,3] = elements['LP_slider']
grid[3,4] = widgets.HTML('<b>Post-Switch Constant [S]</b>')
grid[3,5] = elements['S_switch_slider']

grid[4,0] = widgets.HTML('<b>Post-Switch Constant [LS]</b>')
grid[4,1] = elements['LS_switch_slider']
grid[4,2] = widgets.HTML('<b>S -> A Rate Constant</b>')
grid[4,3] = elements['k_SA_slider']
grid[4,4] = widgets.HTML('<b>A -> B Rate Constant</b>')
grid[4,5] = elements['k_AB_slider']

grid[5,0] = widgets.HTML('<b>B -> P Rate Constant</b>')
grid[5,1] = elements['k_BP_slider']
grid[5,2] = elements['button']    

display(grid, out)

Chances are you were able to get an OK, but not excellent fit to the data provided. Now, let's see how things look with a more realistic dataset in terms of quality and resolution:

<div class="alert alert-block alert-info">

**Instructions:** Run the two cells below. Try putting in the same parameters you just fit.

</div>

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This cell loads in the file we'll be using
#----------------------------------------------------------------------------------------------------------------

filename = 'Day03_Exercise_PartialData.csv'

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell generates the interface that we will use for the following exercise
#----------------------------------------------------------------------------------------------------------------

elements = initialize_elements(compare_drop_value = 'With Subsetting')      

elements['button'].on_click(on_button_clicked)        

out = widgets.Output()                                 

grid = widgets.GridspecLayout(6, 6)                    

grid[0,0] = widgets.HTML('<b>Total Time</b>')
grid[0,1] = elements['total_time_slider']
grid[0,2] = widgets.HTML('<b>Timestep Size</b>')
grid[0,3] = elements['timestep_slider']
grid[0,4] = widgets.HTML('<b>Switch Point</b>')
grid[0,5] = elements['switch_point_slider']

grid[1,0] = widgets.HTML('<b>Pre-Switch Constant [S]</b>')
grid[1,1] = elements['S_slider']
grid[1,2] = widgets.HTML('<b>Initial [A]</b>')
grid[1,3] = elements['A_slider']
grid[1,4] = widgets.HTML('<b>Initial [B]</b>')
grid[1,5] = elements['B_slider']

grid[2,0] = widgets.HTML('<b>Initial [P]</b>')
grid[2,1] = elements['P_slider']
grid[2,2] = widgets.HTML('<b>Pre-Switch Constant [LS]</b>')
grid[2,3] = elements['LS_slider']
grid[2,4] = widgets.HTML('<b>Initial [LA]</b>')
grid[2,5] = elements['LA_slider']

grid[3,0] = widgets.HTML('<b>Initial [LB]</b>')
grid[3,1] = elements['LB_slider']
grid[3,2] = widgets.HTML('<b>Initial [LP]</b>')
grid[3,3] = elements['LP_slider']
grid[3,4] = widgets.HTML('<b>Post-Switch Constant [S]</b>')
grid[3,5] = elements['S_switch_slider']

grid[4,0] = widgets.HTML('<b>Post-Switch Constant [LS]</b>')
grid[4,1] = elements['LS_switch_slider']
grid[4,2] = widgets.HTML('<b>S -> A rate constant</b>')
grid[4,3] = elements['k_SA_slider']
grid[4,4] = widgets.HTML('<b>A -> B rate constant</b>')
grid[4,5] = elements['k_AB_slider']

grid[5,0] = widgets.HTML('<b>B -> P rate constant</b>')
grid[5,1] = elements['k_BP_slider']
grid[5,2] = elements['button']    

display(grid, out)

<div class="alert alert-block alert-success">
    <b>Discussion:</b> How does the addition of noise change things? Discuss within your group. Do you think the effect of noise on your level of certainty would still be there if, instead of fitting by eye, you were using some kind of algorithm/software to do the fitting for you? Once you are done discussing in your group, join the rest of the class for a whole class discussion. 
</div>

<div class="alert alert-block alert-warning"> 
    <b>Break Time</b> Once all groups have finished discussing their findings, we'll take a fifteen minute break.

# **(4.0)** Reversible Michaelis-Menten Kinetics

Similar to first-order kinetics, we can extend Michaelis-Menten kinetics to include reversibility. For the following exercises, we'll once again consider the following pathway with reversible kinetics:

![alt text](MCA_Network.png "Title")

Where the net fluxes **Jsa**, **Jab**, **Jbp**, and **Jpx** can now be described as so (note that Jpx is irreversible, so we use the regular Michaelis-Menten expression for it:

$$ J_{sa} = \frac{Vmax_{SA}^{f}\frac{[S]}{Km_{SA}^{f}}-Vmax_{SA}^{b}\frac{[A]}{Km_{SA}^{b}}}{1+\frac{[S]}{Km_{SA}^{f}}+\frac{[A]}{Km_{SA}^{b}}} $$

$$ J_{ab} = \frac{Vmax_{AB}^{f}\frac{[A]}{Km_{AB}^{f}}-Vmax_{AB}^{b}\frac{[B]}{Km_{AB}^{b}}}{1+\frac{[A]}{Km_{AB}^{f}}+\frac{[B]}{Km_{AB}^{b}}} $$

$$ J_{bp} = \frac{Vmax_{BP}^{f}\frac{[B]}{Km_{BP}^{f}}-Vmax_{BP}^{b}\frac{[P]}{Km_{BP}^{b}}}{1+\frac{[B]}{Km_{BP}^{f}}+\frac{[P]}{Km_{BP}^{b}}} $$

$$ J_{px} = \frac{Vmax_{PX}[P]}{Km_{PX}+[P]} $$

<div class="alert alert-block alert-info">

**Instructions:** Using the cells below, run the simulation and experiment with reversible Michaelis-Menten kinetics.
</div>

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------
# In this cell we are defining functions that we will need during our simulation, as well as the function that carries
# out the simulation itself.
#-----------------------------------------------------------------------------------------------------------------------

def evaluate_Flux_revMM(Vmax_f, Vmax_b, Km_f, Km_b, S, P):         # Defining a new function to evaluate fluxes given parameters
                                                                   # for reversible Michaelis-Menten kinetics
    '''                                                            
    DESCRIPTION
    
    Takes a rate constant k and a substrate concentration S 
    and returns a flux value.
    
    INPUTS
    
    Vmax_f = A numeric value for the Vmax of the forward reaction
    
    Km_f = A numeric value for the Km of the forward reaction
    
    Vmax_b = A numeric value for the Vmax of the reverse reaction
    
    Km_b = A numeric value for the Km of the reverse reaction
    
    S = a numeric concentration of a substrate
    
    P = a numeric concentration of a product
    
    OUTPUTS
    
    flux = flux, or reaction rate, given substrate 
    concentration and a first order rate constant
    '''

    flux = (Vmax_f * (S / Km_f) - Vmax_b * (P / Km_b)) / (1 + (S / Km_f) + (P / Km_b))    # This is where we actually calculate the flux,
                                                                                          # following the equation given in the section 
                                                                                          # (4.0) Markdown cell
    return flux                                                                


def evaluate_dLAdt(Jsa, Jab, f_s, f_a, f_b):                                                                                      
                                                                  
    if Jsa >= 0 and Jab >= 0:
        dLAdt = f_s * Jsa - Jab * f_a
    elif Jsa < 0 and Jab >=0:
        dLAdt = f_a * Jsa - Jab * f_a
    elif Jsa >= 0 and Jab < 0:
        dLAdt = f_s * Jsa - Jab * f_b
    elif Jsa < 0 and Jab < 0:
        dLAdt = f_a * Jsa - Jab * f_b
    return dLAdt

def evaluate_dLBdt(Jab, Jbp, f_a, f_b, f_p):
    if Jab >= 0 and Jbp >= 0:
        dLAdt = f_a * Jab - Jbp * f_b
    elif Jab < 0 and Jbp >=0:
        dLAdt = f_b * Jab - Jbp * f_b
    elif Jab >= 0 and Jbp < 0:
        dLAdt = f_a * Jab - Jbp * f_p
    elif Jab < 0 and Jbp < 0:
        dLAdt = f_b * Jab - Jbp * f_p
    return dLAdt


def evaluate_dLPdt(Jbp, f_b, f_p):
    if Jbp >= 0: 
        dLPdt = Jbp * f_b
    elif Jbp < 0: 
        dLPdt = Jbp * f_p
    return dLPdt

def evaluate_fractional_labeling(current_labeled, current_total): 
    if current_total == 0:                                       
        f = 0                                                             
    else:                                                        
        f = current_labeled / current_total                       
    return f

#-----------------------------------------------------------------------------------------------------------------------
# Now that we have all of these functions defined, we can define the function that will actually run our simulation
# and give us the results we're going to analyze. 
#-----------------------------------------------------------------------------------------------------------------------


def kinetic_simulation_revMM(total_time = 50, timestep = 0.01, switch_point = 20, S = 0, A = 0, B = 0,  
        P = 0, LS = 100, LA = 0, LB = 0, LP = 0, S_switch = 0, LS_switch = 0, Vmax_f_SA = 10, Vmax_b_SA = 1, Km_f_SA = 10, Km_b_SA = 100,
        Vmax_f_AB = 10, Vmax_b_AB = 1, Km_f_AB = 10, Km_b_AB = 100, Vmax_f_BP = 10, Vmax_b_BP = 1, Km_f_BP = 10, Km_b_BP = 100, export = False,
        compare = 'None', filename = '',
        subset = False): 
    
    '''
    DESCRIPTION
    
    A function that carries out a metabolic modeling simulation using first-order kinetics.
    Accepts arguments to both export generated data and import another group's data to 
    compare with simulation results. Can also subset generated data and add random noise
    before exporting.
    
    INPUTS
    
    total_time = A numeric value with arbitrary units that determines how long to run
    the simulation for.
    
    timestep = A numeric value representing how much to increment the time value each
    loop of the simulation.
    
    switch_point = A numeric value representing at which point to switch from an initial
    concentration of labeled and unlabeled S to a "post-switch" value for these same
    quantities
    
    S = A numeric value representing the set concentration of unlabeled S pre-switch
    
    A = A numeric value representing the initial concentration of unlabeled A
    
    B = A numeric value representing the initial concentration of unlabeled B
    
    P = A numeric value representing the initial concentration of unlableed P
    
    LA = A numeric value representing the initial concentration of labeled A
    
    LB = A numeric value representing the initial concentration of labeled B
    
    LP = A numeric value representing the initial concentration of labeled P
    
    S_switch = A numeric value representing the set concentration of unlabeled 
    S post-switch
    
    LS_switch = A numeric value representing the set concentration of labeled 
    S post-switch    
    
    Vmax_f_SA = A numeric value for the Vmax of the forward reaction of S <=> A
    
    Km_f_SA = A numeric value for the Km of the forward reaction of S <=> A
    
    Vmax_b_SA = A numeric value for the Vmax of the reverse reaction of S <=> A
    
    Km_b_SA = A numeric value for the Km of the reverse reaction of S <=> A
    
    Vmax_f_AB = A numeric value for the Vmax of the forward reaction of A <=> B
    
    Km_f_AB = A numeric value for the Km of the forward reaction of A <=> B
    
    Vmax_b_AB = A numeric value for the Vmax of the reverse reaction of A <=> B
    
    Km_b_AB = A numeric value for the Km of the reverse reaction of A <=> B
    
    Vmax_f_BP = A numeric value for the Vmax of the forward reaction of B <=> P
    
    Km_f_BP = A numeric value for the Km of the forward reaction of B <=> P
    
    Vmax_b_BP = A numeric value for the Vmax of the reverse reaction of B <=> P
    
    Km_b_BP = A numeric value for the Km of the reverse reaction of B <=> P
    
    export = A boolean value that determines whether or not to export the 
    simulated data as a .csv file
    
    compare = A boolean value that determines whether or not to load in
    another group's data from a .csv file and plot it out
    
    filename = A string corresponding to the name of the file loaded in
    if compare == True
    
    subset = A boolean value that determines whether or not to subset and
    add noise to the simulated data before exporting (only matters if 
    export == True)
    
    OUTPUTS
    
    None
    
    '''
    
    times = np.arange(0, total_time, timestep)      
    
    S_total = S + LS                               
    A_total = A + LA                             
    B_total = B + LB
    P_total = P + LP
    
    S_list = [S]                                                                                     
    A_list = [A]                                                                                   
    B_list = [B]                                    
    P_list = [P]                                   
    
    LS_list = [LS]                                
    LA_list = [LA]
    LB_list = [LB]
    LP_list = [LP]
        
    S_total_list = [S_total]                       
    A_total_list = [A_total]
    B_total_list = [B_total]
    P_total_list = [P_total]

    fs_list = [evaluate_fractional_labeling(LS, S_total)] 
    fa_list = [evaluate_fractional_labeling(LA, A_total)] 
    fb_list = [evaluate_fractional_labeling(LB, B_total)] 
    fp_list = [evaluate_fractional_labeling(LP, P_total)]
        
    jsa_list = [evaluate_Flux_revMM(Vmax_f_SA, Vmax_b_SA, Km_f_SA, Km_b_SA, S_total_list[-1], A_total_list[-1])] # We use the evaluate_Flux function for 
    jab_list = [evaluate_Flux_revMM(Vmax_f_AB, Vmax_b_AB, Km_f_AB, Km_b_AB, A_total_list[-1], B_total_list[-1])] # reversible Michaelis-Menten kinetics here
    jbp_list = [evaluate_Flux_revMM(Vmax_f_BP, Vmax_b_BP, Km_f_BP, Km_b_BP, B_total_list[-1], P_total_list[-1])]   
    
    for i in times:                                 
        if i >= switch_point:                                                                
            current_S = S_switch                    
            current_LS = LS_switch
        else:                                     
            current_S = S
            current_LS = LS
   
        current_Jsa = evaluate_Flux_revMM(Vmax_f_SA, Vmax_b_SA, Km_f_SA, Km_b_SA, S_total_list[-1], A_total_list[-1]) # We pass the Vmax and Km values for the forward and
        current_Jab = evaluate_Flux_revMM(Vmax_f_AB, Vmax_b_AB, Km_f_AB, Km_b_AB, A_total_list[-1], B_total_list[-1]) # reverse reactions along with substrate and product concentrations
        current_Jbp = evaluate_Flux_revMM(Vmax_f_BP, Vmax_b_BP, Km_f_BP, Km_b_BP, B_total_list[-1], P_total_list[-1]) # to calculate each of the fluxes

        current_A = A_list[-1] + (evaluate_dLAdt(current_Jsa, current_Jab, (1 - fs_list[-1]), (1 - fa_list[-1]), (1 - fb_list[-1])) * timestep)
        
        current_B = B_list[-1] + (evaluate_dLBdt(current_Jab, current_Jbp, (1 - fa_list[-1]), (1 - fb_list[-1]), (1 - fp_list[-1])) * timestep)
        
        current_P = P_list[-1] + (evaluate_dLPdt(current_Jbp, (1 - fb_list[-1]), (1 - fp_list[-1])) * timestep)
        
        current_LA = LA_list[-1] + (evaluate_dLAdt(current_Jsa, current_Jab, fs_list[-1], fa_list[-1], fb_list[-1]) * timestep)
        
        current_LB = LB_list[-1] + (evaluate_dLBdt(current_Jab, current_Jbp, fa_list[-1], fb_list[-1], fp_list[-1]) * timestep)
        
        current_LP = LP_list[-1] + (evaluate_dLPdt(current_Jbp, fb_list[-1], fp_list[-1]) * timestep)
        
        current_S_total = current_S + current_LS
        current_A_total = current_A + current_LA
        current_B_total = current_B + current_LB
        current_P_total = current_P + current_LP
        
        current_fs = evaluate_fractional_labeling(current_LS, current_S_total)
        current_fa = evaluate_fractional_labeling(current_LA, current_A_total)
        current_fb = evaluate_fractional_labeling(current_LB, current_B_total)
        current_fp = evaluate_fractional_labeling(current_LP, current_P_total)
        
        LS_list.append(current_LS)                     
        LA_list.append(current_LA)                     
        LB_list.append(current_LB)
        LP_list.append(current_LP)

        S_list.append(current_S)
        A_list.append(current_A)
        B_list.append(current_B)
        P_list.append(current_P)

        S_total_list.append(current_S_total)
        A_total_list.append(current_A_total)
        B_total_list.append(current_B_total)
        P_total_list.append(current_P_total)

        fs_list.append(current_fs)
        fa_list.append(current_fa)
        fb_list.append(current_fb)
        fp_list.append(current_fp)
                
        jsa_list.append(current_Jsa)
        jab_list.append(current_Jab)
        jbp_list.append(current_Jbp)   
    
    times_including_initial = np.insert(times, 0, times[0] - timestep) 
    
    if compare == 'None':                                           
        
        fig, ax = plt.subplots(1, 3, figsize = (16, 5))             
        ax = ax.flatten()
                
        ax[0].plot(times_including_initial, S_total_list, label = 'S') 
        ax[0].plot(times_including_initial, A_total_list, label = 'A')  
        ax[0].plot(times_including_initial, B_total_list, label = 'B') 
        ax[0].plot(times_including_initial, P_total_list, label = 'P')  
        ax[0].legend()                                                  
        ax[0].set_title('Total Concentration Over Time',                
                        fontweight = 'bold')
        ax[0].set_xlabel('Time', fontweight = 'bold')                   
        ax[0].set_ylabel('Total Concentration', fontweight = 'bold')    

        ax[1].plot(times_including_initial, jsa_list, label = 'S -> A') 
        ax[1].plot(times_including_initial, jab_list, label = 'A -> B') 
        ax[1].plot(times_including_initial, jbp_list, label = 'B -> P')
        ax[1].legend()                                                  
        ax[1].set_title('Fluxes', fontweight = 'bold')                 
        ax[1].set_xlabel('Time', fontweight = 'bold')                   
        ax[1].set_ylabel('Flux', fontweight = 'bold')                   

        ax[2].plot(times_including_initial, fs_list, label = 'S')       
        ax[2].plot(times_including_initial, fa_list, label = 'A')       
        ax[2].plot(times_including_initial, fb_list, label = 'B')       
        ax[2].plot(times_including_initial, fp_list, label = 'P')     
        ax[2].legend()                                                 
        ax[2].set_title('Fraction Labeled Over Time',                  
                        fontweight='bold')
        ax[2].set_xlabel('Time',fontweight='bold')                      
        ax[2].set_ylabel('Total Concentration', fontweight = 'bold')   
            
    elif compare == 'No Subsetting':                                   
                
        other_data = pd.read_csv(filename)                          

        fig, ax = plt.subplots(2, 3, figsize = (16, 10))           
        ax = ax.flatten()
        
        ax[0].plot(times_including_initial, S_total_list, label = 'S') 
        ax[0].plot(times_including_initial, A_total_list, label = 'A') 
        ax[0].plot(times_including_initial, B_total_list, label = 'B')
        ax[0].plot(times_including_initial, P_total_list, label = 'P') 
        ax[0].legend()
        ax[0].set_title("Concentration Over Time", fontweight = 'bold')
        ax[0].set_xlabel('Time', fontweight = 'bold')
        ax[0].set_ylabel('Total Concentration', fontweight = 'bold')
        
        ax[1].plot(times_including_initial, jsa_list, label = 'S -> A') 
        ax[1].plot(times_including_initial, jab_list, label = 'A -> B') 
        ax[1].plot(times_including_initial, jbp_list, label = 'B -> P')  
        ax[1].legend()
        ax[1].set_title("Fluxes Over Time", fontweight = 'bold')
        ax[1].set_xlabel('Time', fontweight = 'bold')
        ax[1].set_ylabel('Flux', fontweight = 'bold')
        
        ax[2].plot(times_including_initial, fs_list, label = 'S') 
        ax[2].plot(times_including_initial, fa_list, label = 'A')
        ax[2].plot(times_including_initial, fb_list, label = 'B')
        ax[2].plot(times_including_initial, fp_list, label = 'P')
        ax[2].legend()
        ax[2].set_title("Fraction Labeled Over Time", fontweight = 'bold')
        ax[2].set_xlabel('Time', fontweight = 'bold')
        ax[2].set_ylabel('Total Concentration', fontweight = 'bold')
    
        ax[3].scatter(other_data['Times'], other_data['Total_S'], label = 'S') 
        ax[3].scatter(other_data['Times'], other_data['Total_A'], label = 'A') 
        ax[3].scatter(other_data['Times'], other_data['Total_B'], label = 'B') 
        ax[3].scatter(other_data['Times'], other_data['Total_P'], label = 'P')     
        ax[3].legend()
        ax[3].set_title("Other Group's Concentration Over Time", fontweight = 'bold')
        ax[3].set_xlabel('Time', fontweight = 'bold')
        ax[3].set_ylabel('Total Concentration', fontweight = 'bold')
                
        ax[4].scatter(other_data['Times'], other_data['SA_Flux'], label = 'S -> A') 
        ax[4].scatter(other_data['Times'], other_data['AB_Flux'], label = 'A -> B')
        ax[4].scatter(other_data['Times'], other_data['BP_Flux'], label = 'B -> P')  
        ax[4].legend()
        ax[4].set_title("Other Group's Fluxes Over Time", fontweight = 'bold')
        ax[4].set_xlabel('Time', fontweight = 'bold')
        ax[4].set_ylabel('Flux', fontweight = 'bold')
                
        ax[5].scatter(other_data['Times'], other_data['fs'], label = 'S', alpha = 0.5) 
        ax[5].scatter(other_data['Times'], other_data['fa'], label = 'A', alpha = 0.5) 
        ax[5].scatter(other_data['Times'], other_data['fb'], label = 'B', alpha = 0.5) 
        ax[5].scatter(other_data['Times'], other_data['fp'], label = 'P', alpha = 0.5)     
        ax[5].legend()
        ax[5].set_title("Other Group's Fraction Labeled Over Time", fontweight = 'bold')
        ax[5].set_xlabel('Time', fontweight = 'bold')
        ax[5].set_ylabel('Total Concentration', fontweight = 'bold')

    elif compare == 'With Subsetting':                                                  
                
        other_data = pd.read_csv(filename)                         

        fig, ax = plt.subplots(1, 3, figsize=(16, 5))               
        ax = ax.flatten()
        
        ax[0].plot(times_including_initial, S_total_list, label = 'S') 
        ax[0].plot(times_including_initial, A_total_list, label = 'A') 
        ax[0].plot(times_including_initial, B_total_list, label = 'B')
        ax[0].plot(times_including_initial, P_total_list, label = 'P')
        ax[0].scatter(other_data['Times'], other_data['Total_S'], label = 'S')  
        ax[0].scatter(other_data['Times'], other_data['Total_A'], label = 'A') 
        ax[0].scatter(other_data['Times'], other_data['Total_B'], label = 'B') 
        ax[0].scatter(other_data['Times'], other_data['Total_P'], label = 'P')             
        ax[0].legend()
        ax[0].set_title("Concentration Over Time", fontweight = 'bold')
        ax[0].set_xlabel('Time', fontweight = 'bold')
        ax[0].set_ylabel('Total Concentration', fontweight = 'bold')
        
        ax[1].plot(times_including_initial, jsa_list, label = 'S -> A') 
        ax[1].plot(times_including_initial, jab_list, label = 'A -> B') 
        ax[1].plot(times_including_initial, jbp_list, label = 'B -> P')
        ax[1].scatter(other_data['Times'], other_data['SA_Flux'], label = 'S -> A') 
        ax[1].scatter(other_data['Times'], other_data['AB_Flux'], label = 'A -> B') 
        ax[1].scatter(other_data['Times'], other_data['BP_Flux'], label = 'B -> P')          
        ax[1].legend()
        ax[1].set_title("Fluxes Over Time", fontweight = 'bold')
        ax[1].set_xlabel('Time', fontweight = 'bold')
        ax[1].set_ylabel('Flux', fontweight = 'bold')
        
        ax[2].plot(times_including_initial, fs_list, label = 'S') 
        ax[2].plot(times_including_initial, fa_list, label = 'A') 
        ax[2].plot(times_including_initial, fb_list, label = 'B')
        ax[2].plot(times_including_initial, fp_list, label = 'P')
        ax[2].scatter(other_data['Times'], other_data['fs'], label = 'S', alpha = 0.5) 
        ax[2].scatter(other_data['Times'], other_data['fa'], label = 'A', alpha = 0.5) 
        ax[2].scatter(other_data['Times'], other_data['fb'], label = 'B', alpha = 0.5) 
        ax[2].scatter(other_data['Times'], other_data['fp'], label = 'P', alpha = 0.5) 
        ax[2].legend()
        ax[2].set_title("Fraction Labeled Over Time", fontweight = 'bold')
        ax[2].set_xlabel('Time', fontweight = 'bold')
        ax[2].set_ylabel('Total Concentration', fontweight = 'bold')
        
    elif compare == 'Only SA':                                    
                
        other_data = pd.read_csv(filename)                          

        fig, ax = plt.subplots(1, 2, figsize = (10, 5))                
        ax = ax.flatten()
        
        ax[0].plot(times_including_initial, S_total_list, label = 'S')
        ax[0].plot(times_including_initial, A_total_list, label = 'A') 
        ax[0].scatter(other_data['Times'], other_data['Total_S'], label = 'S') 
        ax[0].scatter(other_data['Times'], other_data['Total_A'], label = 'A')    
        ax[0].legend()
        ax[0].set_title("Concentration Over Time", fontweight = 'bold')
        ax[0].set_xlabel('Time', fontweight = 'bold')
        ax[0].set_ylabel('Total Concentration', fontweight = 'bold')
        
        ax[1].plot(times_including_initial, fs_list, label = 'S')
        ax[1].plot(times_including_initial, fa_list, label = 'A') 
        ax[1].scatter(other_data['Times'], other_data['fs'], label = 'S', alpha = 0.5) 
        ax[1].scatter(other_data['Times'], other_data['fa'], label = 'A', alpha = 0.5) 
        ax[1].legend()
        ax[1].set_title("Fraction Labeled Over Time", fontweight = 'bold')
        ax[1].set_xlabel('Time', fontweight = 'bold')
        ax[1].set_ylabel('Total Concentration', fontweight = 'bold')     

    elif compare == 'Only BP':                                      
        
        other_data = pd.read_csv(filename)                         

        fig, ax = plt.subplots(1, 2, figsize = (10, 5))                 
        ax = ax.flatten()
        
        ax[0].plot(times_including_initial, B_total_list, label = 'B') 
        ax[0].plot(times_including_initial, P_total_list, label = 'P') 
        ax[0].scatter(other_data['Times'], other_data['Total_B'], label = 'B')  
        ax[0].scatter(other_data['Times'], other_data['Total_P'], label = 'P')       
        ax[0].legend()
        ax[0].set_title("Concentration Over Time", fontweight = 'bold')
        ax[0].set_xlabel('Time', fontweight = 'bold')
        ax[0].set_ylabel('Total Concentration', fontweight = 'bold')
        
        ax[1].plot(times_including_initial, fb_list, label = 'B') 
        ax[1].plot(times_including_initial, fp_list, label = 'P') 
        ax[1].scatter(other_data['Times'], other_data['fb'], label = 'B', alpha = 0.5) 
        ax[1].scatter(other_data['Times'], other_data['fp'], label = 'P', alpha = 0.5) 
        ax[1].legend()
        ax[1].set_title("Fraction Labeled Over Time", fontweight = 'bold')
        ax[1].set_xlabel('Time', fontweight = 'bold')
        ax[1].set_ylabel('Total Concentration', fontweight = 'bold')         
    
    if export:                                                            
        if not subset:                                                       
            results_dictionary = {'Times':times_including_initial,        
                                  'Total_S':S_total_list,                 
                                  'Total_A':A_total_list,
                                  'Total_B':B_total_list,
                                  'Total_P':P_total_list,
                                  'SA_Flux':jsa_list,
                                  'AB_Flux':jab_list,
                                  'BP_Flux':jbp_list,
                                  'fs':fs_list,
                                  'fa':fa_list,
                                  'fb':fb_list,
                                  'fp':fp_list}
            results_df = pd.DataFrame(results_dictionary)                 
            results_df.to_csv('GroupX_FirstOrderData.csv')                
        
        elif subset:                                                    
            
            times_numpy = np.array(times_including_initial)[0::100]       
            S_total_numpy = np.array(S_total_list)[0::100]                
            A_total_numpy = np.array(A_total_list)[0::100]                
            B_total_numpy = np.array(B_total_list)[0::100]
            P_total_numpy = np.array(P_total_list)[0::100]
            jsa_numpy = np.array(jsa_list)[0::100]
            jab_numpy = np.array(jab_list)[0::100]
            jbp_numpy = np.array(jbp_list)[0::100]
            fs_numpy = np.array(fs_list)[0::100]
            fa_numpy = np.array(fa_list)[0::100]
            fb_numpy = np.array(fb_list)[0::100]
            fp_numpy = np.array(fp_list)[0::100]  
                
            S_total_noise = np.random.normal(0,1,size=S_total_numpy.shape)
            S_total_noise = S_total_noise * 0.02 * S_total_numpy          
            S_noisy_array = S_total_numpy + S_total_noise                 

            A_total_noise = np.random.normal(0,1,size=A_total_numpy.shape)
            A_total_noise = A_total_noise * 0.02 * A_total_numpy
            A_noisy_array = A_total_numpy + A_total_noise

            B_total_noise = np.random.normal(0,1,size=B_total_numpy.shape)            
            B_total_noise = B_total_noise * 0.02 * B_total_numpy
            B_noisy_array = B_total_numpy + B_total_noise

            P_total_noise = np.random.normal(0,1,size=P_total_numpy.shape)            
            P_total_noise = P_total_noise * 0.02 * P_total_numpy
            P_noisy_array = P_total_numpy + P_total_noise

            jsa_noise = np.random.normal(0,1,size=jsa_numpy.shape)        
            jsa_noise = jsa_noise * 0.02 * jsa_numpy                      
            jsa_noisy_array = jsa_numpy + jsa_noise
            
            jab_noise = np.random.normal(0,1,size=jab_numpy.shape)        
            jab_noise = jab_noise * 0.02 * jab_numpy
            jab_noisy_array = jab_numpy + jab_noise
            
            jbp_noise = np.random.normal(0,1,size=jbp_numpy.shape)      
            jbp_noise = jbp_noise * 0.02 * jbp_numpy
            jbp_noisy_array = jbp_numpy + jbp_noise
            
            fs_noise = np.random.normal(0,1,size=fs_numpy.shape)        
            fs_noise = fs_noise * 0.02 * fs_numpy                        
            fs_noisy_array = fs_numpy + fs_noise

            fa_noise = np.random.normal(0,1,size=fa_numpy.shape)          
            fa_noise = fa_noise * 0.02 * fa_numpy
            fa_noisy_array = fa_numpy + fa_noise

            fb_noise = np.random.normal(0,1,size=fb_numpy.shape)         
            fb_noise = fb_noise * 0.02 * fb_numpy
            fb_noisy_array = fb_numpy + fb_noise

            fp_noise = np.random.normal(0,1,size=fp_numpy.shape)         
            fp_noise = fp_noise * 0.02 * fp_numpy
            fp_noisy_array = fp_numpy + fp_noise
            
            results_dictionary = {'Times':times_numpy,                    
                                  'Total_S':S_noisy_array,              
                                  'Total_A':A_noisy_array,
                                  'Total_B':B_noisy_array,
                                  'Total_P':P_noisy_array,
                                  'SA_Flux':jsa_noisy_array,
                                  'AB_Flux':jab_noisy_array,
                                  'BP_Flux':jbp_noisy_array,
                                  'fs':fs_noisy_array,
                                  'fa':fa_noisy_array,
                                  'fb':fb_noisy_array,
                                  'fp':fp_noisy_array}
            results_df = pd.DataFrame(results_dictionary)
            results_df.to_csv('GroupX_FirstOrderData_Subsampled.csv')
    
    return fig, ax

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell defines the functions we need for our interface. 
#----------------------------------------------------------------------------------------------------------------

filename = ''

def initialize_elements_revMM(total_time_slider_value = 100, timestep_slider_value = 0.1,                   
                       switch_point_slider_value = 20, S_slider_value = 0, A_slider_value = 0,
                       B_slider_value = 0, P_slider_value = 0, LS_slider_value = 100,
                       LA_slider_value = 0, LB_slider_value = 0, LP_slider_value = 0,
                       S_switch_slider_value = 0, LS_switch_slider_value = 0,
                       Vmax_SA_f_slider_value = 10, Vmax_SA_b_slider_value = 1, Km_SA_f_slider_value = 10, Km_SA_b_slider_value = 100,
                       Vmax_AB_f_slider_value = 10, Vmax_AB_b_slider_value = 1, Km_AB_f_slider_value = 10, Km_AB_b_slider_value = 100,
                       Vmax_BP_f_slider_value = 10, Vmax_BP_b_slider_value = 1, Km_BP_f_slider_value = 10, Km_BP_b_slider_value = 100,
                       compare_drop_value = 'None', subsample_drop_value = False, export_drop_value = False):

    dictionary_of_elements = {'total_time_slider': widgets.IntSlider(min = 1, max = 1000, step = 1, value = total_time_slider_value),
                              'timestep_slider': widgets.FloatSlider(min = 0.01, max = 1, step = 0.01, value = timestep_slider_value),
                              'switch_point_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = switch_point_slider_value),
                              'S_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = S_slider_value),
                              'A_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = A_slider_value),
                              'B_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = B_slider_value),
                              'P_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = P_slider_value),
                              'LS_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LS_slider_value),
                              'LA_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LA_slider_value),
                              'LB_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LB_slider_value),
                              'LP_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LP_slider_value),
                              'S_switch_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = S_switch_slider_value),
                              'LS_switch_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LS_switch_slider_value),
                              'Vmax_SA_f_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Vmax_SA_f_slider_value),
                              'Vmax_SA_b_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Vmax_SA_b_slider_value),
                              'Vmax_AB_f_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Vmax_AB_f_slider_value),
                              'Vmax_AB_b_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Vmax_AB_b_slider_value),
                              'Vmax_BP_f_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Vmax_BP_f_slider_value),
                              'Vmax_BP_b_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Vmax_BP_b_slider_value),
                              'Km_SA_f_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Km_SA_f_slider_value),
                              'Km_SA_b_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Km_SA_b_slider_value),
                              'Km_AB_f_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Km_AB_f_slider_value),
                              'Km_AB_b_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Km_AB_b_slider_value),
                              'Km_BP_f_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Km_BP_f_slider_value),
                              'Km_BP_b_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Km_BP_b_slider_value),
                              'compare_drop': widgets.Dropdown(options = [('None', 'None'), ('No Subsetting', 'No Subsetting'),
                                                                          ('With Subsetting', 'With Subsetting'), ('Only SA', 'Only SA'),
                                                                          ('Only BP', 'Only BP')], value = compare_drop_value),
                              'subsample_drop': widgets.Dropdown(options = [('Yes', True), ('No', False)], value = subsample_drop_value),
                              'export_drop': widgets.Dropdown(options = [('Yes', True), ('No', False)], value = export_drop_value),
                              'button': widgets.Button(description = 'Run')}
    return dictionary_of_elements

def on_button_clicked(_):                           
    with out:
        clear_output()
        kinetic_simulation_revMM_interactive(filename)
        show_inline_matplotlib_plots()
        
def kinetic_simulation_revMM_interactive(filename):                                   
    total_time = elements['total_time_slider'].value
    timestep = elements['timestep_slider'].value
    switch_point = elements['switch_point_slider'].value
    S = elements['S_slider'].value
    A = elements['A_slider'].value
    B = elements['B_slider'].value
    P = elements['P_slider'].value
    LS = elements['LS_slider'].value
    LA = elements['LA_slider'].value
    LB = elements['LB_slider'].value
    LP = elements['LP_slider'].value
    S_switch = elements['S_switch_slider'].value
    LS_switch = elements['LS_switch_slider'].value
    Vmax_SA_f = elements['Vmax_SA_f_slider'].value
    Vmax_SA_b = elements['Vmax_SA_b_slider'].value
    Vmax_AB_f = elements['Vmax_AB_f_slider'].value
    Vmax_AB_b = elements['Vmax_AB_b_slider'].value
    Vmax_BP_f = elements['Vmax_BP_f_slider'].value
    Vmax_BP_b = elements['Vmax_BP_b_slider'].value
    Km_SA_f = elements['Km_SA_f_slider'].value
    Km_SA_b = elements['Km_SA_b_slider'].value
    Km_AB_f = elements['Km_AB_f_slider'].value
    Km_AB_b = elements['Km_AB_b_slider'].value
    Km_BP_f = elements['Km_BP_f_slider'].value
    Km_BP_b = elements['Km_BP_b_slider'].value
    compare = elements['compare_drop'].value
    subsample = elements['subsample_drop'].value
    export = elements['export_drop'].value
    
    kinetic_simulation_revMM(total_time, timestep, switch_point, S, A, B, P, LS, LA, LB,
        LP, S_switch, LS_switch, Vmax_SA_f, Vmax_SA_b, Km_SA_f, Km_SA_b,
              Vmax_AB_f, Vmax_AB_b, Km_AB_f, Km_AB_b, Vmax_BP_f, Vmax_BP_b,
              Km_BP_f, Km_BP_b, export, compare, filename, subsample)

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell generates generates the interface that we will use for the following exercise
#----------------------------------------------------------------------------------------------------------------

elements = initialize_elements_revMM()                 

elements['button'].on_click(on_button_clicked)        
                                                       

out = widgets.Output()                                

grid = widgets.GridspecLayout(10, 6)                  

grid[0,0] = widgets.HTML('<b>Total Time</b>')
grid[0,1] = elements['total_time_slider']
grid[0,2] = widgets.HTML('<b>Timestep Size</b>')
grid[0,3] = elements['timestep_slider']
grid[0,4] = widgets.HTML('<b>Switch Point</b>')
grid[0,5] = elements['switch_point_slider']

grid[1,0] = widgets.HTML('<b>Pre-Switch Constant [S]</b>')
grid[1,1] = elements['S_slider']
grid[1,2] = widgets.HTML('<b>Initial [A]</b>')
grid[1,3] = elements['A_slider']
grid[1,4] = widgets.HTML('<b>Initial [B]</b>')
grid[1,5] = elements['B_slider']

grid[2,0] = widgets.HTML('<b>Initial [P]</b>')
grid[2,1] = elements['P_slider']
grid[2,2] = widgets.HTML('<b>Pre-Switch Constant [LS]</b>')
grid[2,3] = elements['LS_slider']
grid[2,4] = widgets.HTML('<b>Initial [LA]</b>')
grid[2,5] = elements['LA_slider']

grid[3,0] = widgets.HTML('<b>Initial [LB]</b>')
grid[3,1] = elements['LB_slider']
grid[3,2] = widgets.HTML('<b>Initial [LP]</b>')
grid[3,3] = elements['LP_slider']
grid[3,4] = widgets.HTML('<b>Post-Switch Constant [S]</b>')
grid[3,5] = elements['S_switch_slider']

grid[4,0] = widgets.HTML('<b>Post-Switch Constant [LS]</b>')
grid[4,1] = elements['LS_switch_slider']
grid[4,2] = widgets.HTML('<b>S -> A Vmax</b>')
grid[4,3] = elements['Vmax_SA_f_slider']
grid[4,4] = widgets.HTML('<b>S -> A Vmax</b>')
grid[4,5] = elements['Vmax_SA_b_slider']

grid[5,0] = widgets.HTML('<b>S -> A Km</b>')
grid[5,1] = elements['Km_SA_f_slider']
grid[5,2] = widgets.HTML('<b>S -> A Km</b>')
grid[5,3] = elements['Km_SA_b_slider']
grid[5,4] = widgets.HTML('<b>A -> B Vmax</b>')
grid[5,5] = elements['Vmax_AB_f_slider']

grid[6,0] = widgets.HTML('<b>A -> B Vmax</b>')
grid[6,1] = elements['Vmax_AB_b_slider']
grid[6,2] = widgets.HTML('<b>A -> B Km</b>')
grid[6,3] = elements['Km_AB_f_slider']
grid[6,4] = widgets.HTML('<b>A -> B Km</b>')
grid[6,5] = elements['Km_AB_b_slider']

grid[7,0] = widgets.HTML('<b>B -> P Vmax</b>')
grid[7,1] = elements['Vmax_BP_f_slider']
grid[7,2] = widgets.HTML('<b>B -> P Vmax</b>')
grid[7,3] = elements['Vmax_BP_b_slider']
grid[7,4] = widgets.HTML('<b>B -> P Km</b>')
grid[7,5] = elements['Km_BP_f_slider']

grid[8,0] = widgets.HTML('<b>B -> P Km</b>')
grid[8,1] = elements['Km_BP_b_slider']
grid[8,2] = widgets.HTML('<b>Export</b>')
grid[8,3] = elements['export_drop']
grid[8,4] = widgets.HTML('<b>Compare</b>')
grid[8,5] = elements['compare_drop']

grid[9,0] = widgets.HTML('<b>Subsample</b>')
grid[9,1] = elements['subsample_drop']
grid[9,2] = elements['button']    

display(grid,out)

<div class="alert alert-block alert-success">
    <b>Discussion:</b> How do your starting parameters compare to your fitted parameters? How many parameters did you need to change to feel like you got a good fit? How confident are you in the fit? Discuss amongst your group and when everyone's ready, we'll have a whole class discussion.
</div>

# **(5.0)** Response Coefficients

So far, we've discussed Control Coefficients, which describe the sensitivity of pathway flux or metabolite concentration to the concentration or activity of an enzyme in a pathway. We've also discussed elasticities, which describe how the rate of the reaction catalyzed by a particular enzyme varies with substrate and product concentrations. What if we're interested in some **effector** or **inhibitor** that is having some effect on the flux through our pathway, but which is not itself a substrate, product, or enzyme in that pathway? Well, that's where **Response Coefficients** come in. If we have some **effector** or **inhibitor** I that affects the pathway flux **Jpx**, we can define the **Response Coefficient** R as:

$$ R_{[I]}^{Jpx} = \frac{\partial \ln Jpx}{\partial \ln X} $$

Interestingly, we can calculate the **Response Coefficient** directly or as a product of a control coefficient and an elasticity, as shown here.

$$ R_{[I]}^{Jpx} = C_{Vab}^{Jpx} * \epsilon_{[X]}^{Vab} $$

Where this expression can be expanded as follows:

$$ \frac{\partial \ln Jpx}{\partial \ln [I]} = \frac{\partial \ln Jpx}{\partial \ln Vab} * \frac{\partial \ln Vab}{\partial \ln [I]} $$

Conveniently, this definition of **R** holds whatever the precise mechanism is by which the compound **X** affects the flux through your pathway. For the sake of our example, let's say **X** is a noncompetitive inhibitor of the enzyme catalyzing the reaction A &#8594; B. We can define the flux through this reaction (assuming reversible Michaelis-Menten kinetics and noncompetitive inhibition) as:

$$ J_{ab} = \frac{Vmax_{AB}^{f}\frac{[A]}{Km_{AB}^{f}}-Vmax_{AB}^{b}\frac{[B]}{Km_{AB}^{b}}}{1+\frac{[A]}{Km_{AB}^{f}}+\frac{[B]}{Km_{AB}^{b}}+\frac{[I]}{Ki}} $$

Where **[I]** is the concentration of our inhibitor and the term **Ki** defines the concentration of **I** at which 1/2 maximal inhibition of the reaction A &#8594; B is achieved. Note that this expression only tells us the effect of the parameters **I** and **Ki** on the flux through the reaction A &#8594; B in isolation; it doesn't tell us how the sensitivity of flux through our overall pathway to these parameters. Let's use the code below to characterize the value of **R** across a range of parameter values.

<div class="alert alert-block alert-info">

**Instructions:** Using the code blocks below, explore how varying the parameters of the simulation affect the response coefficient.

</div>

In [ ]:
#-----------------------------------------------------------------------------------------------------------------------
# In this cell we define the functions we need to calculate a response coefficient to a noncompetitive inhibitor I. 
# More extensive comments on the auto_MCA() function can be found at the end of the Day02 notebook.
#-----------------------------------------------------------------------------------------------------------------------

def evaluate_Flux_MM(Vmax, Km, S, enzyme):                        
    
    flux = (Vmax * enzyme * S) / (Km + S)                          
            
    return flux                                                    

def evaluate_Flux_revMM(Vmax_f, Vmax_b, Km_f, Km_b, S, P, enzyme):

    flux = (Vmax_f * enzyme * (S / Km_f) - Vmax_b * enzyme * (P / Km_b)) / (1 + (S / Km_f) + (P / Km_b))  
            
    return flux   

def evaluate_Flux_revMM_jab(Vmax_f_AB, Vmax_b_AB, Km_f_AB, Km_b_AB, enzyme_AB, A, B, I, Ki): # We have to calculate Jab's flux differently
                                                                                             # due to the presence of an inhibitor
    Jab = ((Vmax_f_AB * enzyme_AB * (A / Km_f_AB)) - (Vmax_b_AB * enzyme_AB * (B / Km_b_AB))) / (1 + (A / Km_f_AB) + (B / Km_b_AB) + (I / Ki))
    
    return Jab

def evaluate_dLAdt(Jsa, Jab, f_s, f_a, f_b):                      
    
    if Jsa >= 0 and Jab >= 0:
        dLAdt = f_s * Jsa - Jab * f_a
    elif Jsa < 0 and Jab >=0:
        dLAdt = f_a * Jsa - Jab * f_a
    elif Jsa >= 0 and Jab < 0:
        dLAdt = f_s * Jsa - Jab * f_b
    elif Jsa < 0 and Jab < 0:
        dLAdt = f_a * Jsa - Jab * f_b
    return dLAdt

def evaluate_dLBdt(Jab, Jbp, f_a, f_b, f_p):
    if Jab >= 0 and Jbp >= 0:
        dLAdt = f_a * Jab - Jbp * f_b
    elif Jab < 0 and Jbp >=0:
        dLAdt = f_b * Jab - Jbp * f_b
    elif Jab >= 0 and Jbp < 0:
        dLAdt = f_a * Jab - Jbp * f_p
    elif Jab < 0 and Jbp < 0:
        dLAdt = f_b * Jab - Jbp * f_p
    return dLAdt


def evaluate_dLPdt(Jbp, Jpx, f_b, f_p):
    if Jbp >= 0: 
        dLPdt = Jbp * f_b - Jpx * f_p
    elif Jbp < 0: 
        dLPdt = Jbp * f_p - Jpx * f_p
    return dLPdt

def auto_MCA_revMM(total_time = 10000, timestep = 1, S = 10, A = 0, B = 0, P = 0, LS = 0, LA = 0, LB = 0,
        LP = 0, Vmax_f_SA = 10, Vmax_b_SA = 1, Km_f_SA = 10, Km_b_SA = 100, Vmax_f_AB = 10, Vmax_b_AB = 1, Km_f_AB = 10, Km_b_AB = 100,
        Vmax_f_BP = 10, Vmax_b_BP = 1, Km_f_BP = 10, Km_b_BP = 100, Vmax_f_PX = 10, Km_f_PX = 10, enzyme_SA = 1, enzyme_AB = 1, 
                   enzyme_BP=1, enzyme_PX = 1, I = 5, Ki = 10): 
    
    '''
    DESCRIPTION
    
    A function that carries out a metabolic modeling simulation using reversible first-order kinetics.
    Samples flux values for a final pathway flux at steady-state and prints out the results in a plot.
    
    INPUTS
    
    total_time = A numeric value with arbitrary units that determines how long to run
    the simulation for.
    
    timestep = A numeric value representing how much to increment the time value each
    loop of the simulation.
    
    S = A numeric value representing the set concentration of unlabeled S 
    
    A = A numeric value representing the initial concentration of unlabeled A
    
    B = A numeric value representing the initial concentration of unlabeled B
    
    P = A numeric value representing the initial concentration of unlabeled P
    
    I = A numeric value representing the concentration of inhibitor I
    
    LS = A numeric value representing the set concentration of labeled S
    
    LA = A numeric value representing the initial concentration of labeled A
    
    LB = A numeric value representing the initial concentration of labeled B
    
    LP = A numeric value representing the initial concentration of labeled P
    
    Vmax_f_SA = A numeric value for the Vmax of the forward reaction of S <=> A
    
    Km_f_SA = A numeric value for the Km of the forward reaction of S <=> A
    
    Vmax_b_SA = A numeric value for the Vmax of the reverse reaction of S <=> A
    
    Km_b_SA = A numeric value for the Km of the reverse reaction of S <=> A
    
    Vmax_f_AB = A numeric value for the Vmax of the forward reaction of A <=> B
    
    Km_f_AB = A numeric value for the Km of the forward reaction of A <=> B
    
    Vmax_b_AB = A numeric value for the Vmax of the reverse reaction of A <=> B
    
    Km_b_AB = A numeric value for the Km of the reverse reaction of A <=> B
    
    Vmax_f_BP = A numeric value for the Vmax of the forward reaction of B <=> P
    
    Km_f_BP = A numeric value for the Km of the forward reaction of B <=> P
    
    Vmax_b_BP = A numeric value for the Vmax of the reverse reaction of B <=> P
    
    Km_b_BP = A numeric value for the Km of the reverse reaction of B <=> P
    
    Vmax_PX = A numeric value for the Vmax of the reverse reaction of B <=> P
    
    Km_PX = A numeric value for the Km of the reverse reaction of B <=> P
    
    enzyme_SA = A numeric value representing the concentration of the enzyme catalyzing S <=> A

    enzyme_AB = A numeric value representing the concentration of the enzyme catalyzing A <=> B
    
    enzyme_BP = A numeric value representing the concentration of the enzyme catalyzing B <=> P
    
    enzyme_PX = A numeric value representing the concentration of the enzyme catalyzing P -> X

    Ki = A numeric value representing the affinity of the inhibitor I for the enzyme catalyzing A <=> B
    
    OUTPUTS
    
    None
    
    '''
    
    times = np.arange(0,total_time,timestep)                    
    
    enzymes = ['sa', 'ab', 'bp', 'px']                             
    concentrations = np.arange(0.01, 20, 0.01)                     
    fluxes_to_plot = []                                           
    dydx_to_plot = []                                              
    
    jsa_values = []
    jab_values = []
    jbp_values = []
    jpx_values = []

    for concentration in concentrations:                          

        S_total = S + LS                                           
        A_total = A + LA                                           
        B_total = B + LB
        P_total = P + LP

        S_list = [S]                                                                                 
        A_list = [A]                                                                                     
        B_list = [B]                                      
        P_list = [P]                                      

        LS_list = [LS]                                    
        LA_list = [LA]
        LB_list = [LB]
        LP_list = [LP]

        S_total_list = [S_total]                        
        A_total_list = [A_total]
        B_total_list = [B_total]
        P_total_list = [P_total]

        fs_list = [evaluate_fractional_labeling(LS, S_total)]              
        fa_list = [evaluate_fractional_labeling(LA, A_total)]              
        fb_list = [evaluate_fractional_labeling(LB, B_total)]               
        fp_list = [evaluate_fractional_labeling(LP, P_total)]

        jsa_list = [evaluate_Flux_revMM(Vmax_f_SA, Vmax_b_SA, Km_f_SA, Km_b_SA, S_total_list[-1], A_total_list[-1], enzyme_SA)]
        jab_list = [evaluate_Flux_revMM_jab(Vmax_f_AB, Vmax_b_AB, Km_f_AB, Km_b_AB, enzyme_AB, A_total_list[-1], B_total_list[-1], concentration, Ki)]
        jbp_list = [evaluate_Flux_revMM(Vmax_f_BP, Vmax_b_BP, Km_f_BP, Km_b_BP, B_total_list[-1], P_total_list[-1], enzyme_BP)]
        jpx_list = [evaluate_Flux_MM(Vmax_f_BP, Km_f_BP, P_total_list[-1], enzyme_PX)]

        for i in times:                                 

            current_S = S                                
            current_LS = LS                              

            current_Jsa = evaluate_Flux_revMM(Vmax_f_SA, Vmax_b_SA, Km_f_SA, Km_b_SA, S_total_list[-1], A_total_list[-1], enzyme_SA)
            current_Jab = evaluate_Flux_revMM_jab(Vmax_f_AB, Vmax_b_AB, Km_f_AB, Km_b_AB, enzyme_AB, A_total_list[-1], B_total_list[-1], concentration, Ki)
            current_Jbp = evaluate_Flux_revMM(Vmax_f_BP, Vmax_b_BP, Km_f_BP, Km_b_BP, B_total_list[-1], P_total_list[-1], enzyme_BP)
            current_Jpx = evaluate_Flux_MM(Vmax_f_BP, Km_f_BP, P_total_list[-1], enzyme_PX)

            current_A = A_list[-1] + (evaluate_dLAdt(current_Jsa, current_Jab, (1 - fs_list[-1]), (1 - fa_list[-1]), (1 - fb_list[-1])) * timestep)
            current_B = B_list[-1] + (evaluate_dLBdt(current_Jab, current_Jbp, (1 - fa_list[-1]), (1 - fb_list[-1]), (1 - fp_list[-1])) * timestep)
            current_P = P_list[-1] + (evaluate_dLPdt(current_Jbp, current_Jpx, (1 - fb_list[-1]), (1 - fp_list[-1])) * timestep)
            current_LA = LA_list[-1] + (evaluate_dLAdt(current_Jsa, current_Jab, fs_list[-1], fa_list[-1], fb_list[-1]) * timestep)
            current_LB = LB_list[-1]+(evaluate_dLBdt(current_Jab, current_Jbp, fa_list[-1], fb_list[-1], fp_list[-1]) * timestep)
            current_LP = LP_list[-1]+(evaluate_dLPdt(current_Jbp, current_Jpx, fb_list[-1], fp_list[-1]) * timestep)

            current_S_total = current_S + current_LS
            current_A_total = current_A + current_LA
            current_B_total = current_B + current_LB
            current_P_total = current_P + current_LP        
            current_fs = evaluate_fractional_labeling(current_LS, current_S_total)
            current_fa = evaluate_fractional_labeling(current_LA, current_A_total)
            current_fb = evaluate_fractional_labeling(current_LB, current_B_total)
            current_fp = evaluate_fractional_labeling(current_LP, current_P_total)          

            LS_list.append(current_LS)
            LA_list.append(current_LA)
            LB_list.append(current_LB)
            LP_list.append(current_LP)

            S_list.append(current_S)
            A_list.append(current_A)
            B_list.append(current_B)
            P_list.append(current_P)

            S_total_list.append(current_S_total)
            A_total_list.append(current_A_total)
            B_total_list.append(current_B_total)
            P_total_list.append(current_P_total)

            fs_list.append(current_fs)
            fa_list.append(current_fa)
            fb_list.append(current_fb)
            fp_list.append(current_fp)

            jsa_list.append(current_Jsa)
            jab_list.append(current_Jab)
            jbp_list.append(current_Jbp)
            jpx_list.append(current_Jpx)

        jsa_net = np.array(jsa_list)            
        jab_net = np.array(jab_list)           
        jbp_net = np.array(jbp_list)           
        jpx_net = np.array(jpx_list)  

        jsa_values.append(jsa_net[-1])
        jab_values.append(jab_net[-1])
        jbp_values.append(jbp_net[-1])
        jpx_values.append(jpx_net[-1])
        
    spline_jsa = interpolate.splrep(np.log(concentrations), np.log(jsa_values))
    spline_jab = interpolate.splrep(np.log(concentrations), np.log(jab_values))
    spline_jbp = interpolate.splrep(np.log(concentrations), np.log(jbp_values))
    spline_jpx = interpolate.splrep(np.log(concentrations), np.log(jpx_values))


    dydx_jsa = interpolate.splev(np.log(concentrations), spline_jsa, der = 1)
    dydx_jab = interpolate.splev(np.log(concentrations), spline_jab, der = 1)
    dydx_jbp = interpolate.splev(np.log(concentrations), spline_jbp, der = 1)
    dydx_jpx = interpolate.splev(np.log(concentrations), spline_jpx, der = 1)

        
    times_including_initial = np.insert(times, 0, times[0] - timestep) #

    fig, ax = plt.subplots(1, 1)
    
    focus_index = np.where(np.array(concentrations).round(3) == I)


    response_coefficient_jsa = str(round(float(dydx_jsa[focus_index]), 4))
    response_coefficient_jab = str(round(float(dydx_jab[focus_index]), 4))
    response_coefficient_jbp = str(round(float(dydx_jbp[focus_index]), 4))
    response_coefficient_jpx = str(round(float(dydx_jpx[focus_index]), 4))

    ax.plot(np.log(concentrations), np.log(jpx_values), label = 'ln Flux')
    ax.plot(np.log(concentrations), dydx_jpx, label = '1st Derivative')
    ax.set_title('Response Coefficient: {0}'.format(response_coefficient_jpx, 4), fontweight = 'bold')
    ax.legend()
    ax.set_xlabel('Ln Concentration', fontweight = 'bold')
    ax.set_ylabel('Ln Jpx Flux', fontweight = 'bold')
    ax.vlines(np.log(concentrations[focus_index]), -3, 3)


In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell defines the functions we need for our interface. 
#----------------------------------------------------------------------------------------------------------------

filename = ''

def initialize_elements_auto_MCA(total_time_slider_value = 500, timestep_slider_value = 1,                   
                       S_slider_value = 10, A_slider_value = 0, B_slider_value = 0, P_slider_value = 0, I_slider_value = 5, Ki_slider_value = 10,
                       LS_slider_value=0,
                       LA_slider_value=0, LB_slider_value = 0, LP_slider_value = 0,
                       Vmax_SA_f_slider_value = 10, Vmax_SA_b_slider_value = 1, Km_SA_f_slider_value = 10, Km_SA_b_slider_value = 100,
                       Vmax_AB_f_slider_value = 10, Vmax_AB_b_slider_value = 1, Km_AB_f_slider_value = 10, Km_AB_b_slider_value = 100,
                       Vmax_BP_f_slider_value = 10, Vmax_BP_b_slider_value = 1, Km_BP_f_slider_value = 10, Km_BP_b_slider_value = 100,
                       Vmax_PX_slider_value = 10, Km_PX_slider_value = 10,
                       enzyme_SA_slider_value = 0.5, enzyme_AB_slider_value = 0.5, enzyme_BP_slider_value = 0.5,
                       enzyme_PX_slider_value = 0.5):

    dictionary_of_elements = {'total_time_slider': widgets.IntSlider(min = 1, max = 2000, step = 1, value = total_time_slider_value),
                              'timestep_slider': widgets.FloatSlider(min = 0.01, max = 1, step = 0.01, value = timestep_slider_value),
                              'S_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = S_slider_value),
                              'A_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = A_slider_value),
                              'B_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = B_slider_value),
                              'P_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = P_slider_value),
                              'I_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = I_slider_value),
                              'Ki_slider':widgets.IntSlider(min = 0, max = 40, step = 1, value = Ki_slider_value),
                              'LS_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LS_slider_value),
                              'LA_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LA_slider_value),
                              'LB_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LB_slider_value),
                              'LP_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = LP_slider_value),
                              'Vmax_SA_f_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Vmax_SA_f_slider_value),
                              'Vmax_SA_b_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Vmax_SA_b_slider_value),
                              'Km_SA_f_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Km_SA_f_slider_value),
                              'Km_SA_b_slider': widgets.IntSlider(min = 0, max = 1000, step = 1, value = Km_SA_b_slider_value),
                              'Vmax_AB_f_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Vmax_AB_f_slider_value),
                              'Vmax_AB_b_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Vmax_AB_b_slider_value),
                              'Km_AB_f_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Km_AB_f_slider_value),
                              'Km_AB_b_slider': widgets.IntSlider(min = 0, max = 1000, step = 1, value = Km_AB_b_slider_value),
                              'Vmax_BP_f_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Vmax_BP_f_slider_value),
                              'Vmax_BP_b_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Vmax_BP_b_slider_value),
                              'Km_BP_f_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Km_BP_f_slider_value),
                              'Km_BP_b_slider': widgets.IntSlider(min = 0, max = 1000, step = 1, value = Km_BP_b_slider_value),
                              'Vmax_PX_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Vmax_PX_slider_value),
                              'Km_PX_slider': widgets.IntSlider(min = 0, max = 100, step = 1, value = Km_PX_slider_value),
                              'enzyme_SA_slider':widgets.FloatSlider(min = 0.1, max = 1.0, step = 0.01, value = enzyme_SA_slider_value),
                              'enzyme_AB_slider':widgets.FloatSlider(min = 0.1, max = 1.0, step = 0.01, value = enzyme_AB_slider_value),
                              'enzyme_BP_slider':widgets.FloatSlider(min = 0.1, max = 1.0, step = 0.01, value = enzyme_BP_slider_value),
                              'enzyme_PX_slider':widgets.FloatSlider(min = 0.1, max = 1.0, step = 0.01, value = enzyme_PX_slider_value),
                              'button': widgets.Button(description = 'Run')}

    return dictionary_of_elements


def on_button_clicked(_):                          
    with out:
        clear_output()
        auto_MCA_interactive()
        show_inline_matplotlib_plots()
        
def auto_MCA_interactive():                             
    timestep = elements['timestep_slider'].value
    S = elements['S_slider'].value
    A = elements['A_slider'].value
    B = elements['B_slider'].value
    P = elements['P_slider'].value
    I = elements['I_slider'].value
    Ki = elements['Ki_slider'].value
    LS = elements['LS_slider'].value
    LA = elements['LA_slider'].value
    LB = elements['LB_slider'].value
    LP = elements['LP_slider'].value
    Vmax_SA_f = elements['Vmax_SA_f_slider'].value
    Vmax_SA_b = elements['Vmax_SA_b_slider'].value
    Km_SA_f = elements['Km_SA_f_slider'].value
    Km_SA_b = elements['Km_SA_b_slider'].value
    Vmax_AB_f = elements['Vmax_AB_f_slider'].value
    Vmax_AB_b = elements['Vmax_AB_b_slider'].value
    Km_AB_f = elements['Km_AB_f_slider'].value
    Km_AB_b = elements['Km_AB_b_slider'].value
    Vmax_BP_f = elements['Vmax_BP_f_slider'].value
    Vmax_BP_b = elements['Vmax_BP_b_slider'].value
    Km_BP_f = elements['Km_BP_f_slider'].value
    Km_BP_b = elements['Km_BP_b_slider'].value
    Vmax_PX = elements['Vmax_PX_slider'].value
    Km_PX = elements['Km_PX_slider'].value
    enzyme_SA = elements['enzyme_SA_slider'].value
    enzyme_AB = elements['enzyme_AB_slider'].value
    enzyme_BP = elements['enzyme_BP_slider'].value
    enzyme_PX = elements['enzyme_PX_slider'].value

    auto_MCA_revMM(total_time, timestep, S, A, B, P, LS, LA, LB,
        LP, Vmax_SA_f, Vmax_SA_b, Km_SA_f, Km_SA_b,
                        Vmax_AB_f, Vmax_AB_b, Km_AB_f, Km_AB_b,
                        Vmax_BP_f, Vmax_BP_b, Km_BP_f, Km_BP_b,
                        Vmax_PX, Km_PX,
          enzyme_SA, enzyme_AB, enzyme_BP, enzyme_PX, I, Ki)

In [ ]:
#----------------------------------------------------------------------------------------------------------------
# This code cell generates generates the interface that we will use for the following exercise
#----------------------------------------------------------------------------------------------------------------

elements = initialize_elements_auto_MCA()             

elements['button'].on_click(on_button_clicked)         

out = widgets.Output()                                

grid = widgets.GridspecLayout(11,6)                    

grid[0,0] = widgets.HTML('<b>Total Time</b>') 
grid[0,1] = elements['total_time_slider']
grid[0,2] = widgets.HTML('<b>Timestep Size</b>')
grid[0,3] = elements['timestep_slider']
grid[0,4] = widgets.HTML('<b>Constant [S]</b>')
grid[0,5] = elements['S_slider']


grid[1,0] = widgets.HTML('<b>Initial [A]</b>')
grid[1,1] = elements['A_slider']
grid[1,2] = widgets.HTML('<b>Initial [B]</b>')
grid[1,3] = elements['B_slider']
grid[1,4] = widgets.HTML('<b>Initial [P]</b>')
grid[1,5] = elements['P_slider']

grid[2,0] = widgets.HTML('<b>Constant [I]</b>')
grid[2,1] = elements['I_slider']
grid[2,2] = widgets.HTML('<b>Ki</b>')
grid[2,3] = elements['Ki_slider']
grid[2,4] = widgets.HTML('<b>Constant [LS]</b>')
grid[2,5] = elements['LS_slider']

grid[3,0] = widgets.HTML('<b>Initial [LA]</b>')
grid[3,1] = elements['LA_slider']
grid[3,2] = widgets.HTML('<b>Initial [LB]</b>')
grid[3,3] = elements['LB_slider']
grid[3,4] = widgets.HTML('<b>Initial [LP]</b>')
grid[3,5] = elements['LP_slider']

grid[4,0] = widgets.HTML('<b>Vmax S -> A</b>')
grid[4,1] = elements['Vmax_SA_f_slider']
grid[4,2] = widgets.HTML('<b>Vmax A -> S</b>')
grid[4,3] = elements['Vmax_SA_b_slider']
grid[4,4] = widgets.HTML('<b>Km S -> A</b>')
grid[4,5] = elements['Km_SA_f_slider']

grid[5,0] = widgets.HTML('<b>Km A -> S</b>')
grid[5,1] = elements['Km_SA_b_slider']
grid[5,2] = widgets.HTML('<b>Vmax A -> B</b>')
grid[5,3] = elements['Vmax_AB_f_slider']
grid[5,4] = widgets.HTML('<b>Vmax B -> A</b>')
grid[5,5] = elements['Vmax_AB_b_slider']

grid[6,0] = widgets.HTML('<b>Km A -> B</b>')
grid[6,1] = elements['Km_AB_f_slider']
grid[6,2] = widgets.HTML('<b>Km B -> A</b>')
grid[6,3] = elements['Km_AB_b_slider']
grid[6,4] = widgets.HTML('<b>Vmax B -> P</b>')
grid[6,5] = elements['Vmax_BP_f_slider']

grid[7,0] = widgets.HTML('<b>Vmax P -> B</b>')
grid[7,1] = elements['Vmax_BP_b_slider']
grid[7,2] = widgets.HTML('<b>Km B -> P</b>')
grid[7,3] = elements['Km_BP_f_slider']
grid[7,4] = widgets.HTML('<b>Km P -> B</b>')
grid[7,5] = elements['Km_BP_b_slider']

grid[8,0] = widgets.HTML('<b>Vmax P -> X</b>')
grid[8,1] = elements['Vmax_PX_slider']
grid[8,2] = widgets.HTML('<b>Km P -> X</b>')
grid[8,3] = elements['Km_PX_slider']
grid[8,4] = widgets.HTML('<b>[SA Enzyme]</b>')
grid[8,5] = elements['enzyme_SA_slider']

grid[9,0] = widgets.HTML('<b>[AB Enzyme]</b>')
grid[9,1] = elements['enzyme_AB_slider']
grid[9,2] = widgets.HTML('<b>[BP Enzyme]</b>')
grid[9,3] = elements['enzyme_BP_slider']

grid[10,0] = widgets.HTML('<b>[PX Enzyme]</b>')
grid[10,1] = elements['enzyme_PX_slider']
grid[10,2] = elements['button']    

display(grid,out) 

<div class="alert alert-block alert-success">
    <b>Discussion:</b> What leads to high R values? How about low R values? Can you relate conditions leading to high control coefficient values for the reaction A &#8594; B to either high or low R values? Discuss amongst your group and then join a whole class discussion on this.